# To Do:

3. http://mithra.eecs.umich.edu/demo/ranking/
4. Split into multiple files
5. Apply IBM 360 and Aequitas
7. implement this check for demographic parity.... https://www.prevuehr.com/resources/insights/adverse-impact-analysis-four-fifths-rule/

* Add abitity to select the protected attributes to analysis (race/sex/race with sex, etc)
* Add abitity to change the order of the protected attributes for the analysis
* https://towardsdatascience.com/an-easier-way-to-encode-categorical-features-d840ff6b3900 other encoding types.
* add the % value to the bar chart
* add a bubble chart 


* Ask about proxy attributea
* Ask about inputs that may be the output of another algorithm

****Pairwise Ranking vs Pairwise Comparison (individual fairness)


Pandas CrossTab https://pbpython.com/pandas-crosstab.html
Pandas Styles https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html


Improve the heatmap:
https://towardsdatascience.com/better-heatmaps-and-correlation-matrix-plots-in-python-41445d0f2bec

////
Use this as part of the class to print statistics on any dropdown. 
print ("Mimimum FYA:", dp.get_df()['ZFYA'].max())
print ("Maximum FYA:", dp.get_df()['ZFYA'].min())
print ("Maximum FYA:", dp.get_df()['ZFYA'].mean())
Type
Missing Values
Number of unique entries
Other relevant Data>
/////
https://stackoverflow.com/questions/45286696/how-to-display-dataframe-next-to-plot-in-jupyter-notebook/45287049

# About the Law School Admission data
The Law School Admission Council conducted a survey across 163 law schools in the United States

https://www.luciusbynum.com/project/counterfactual-fairness/


It contains information on 21,790 law students such as:
- Entrance exam scores (LSAT)
- Grade-point average (GPA) collected prior to law school,
- sander_index' - Sander proposes combining the LSAT and undergraduate GPA into a single
weighted average, using weights that correspond roughly to those used in many law schools. We refer to this average as the “Sander Index.” Because the units of this index are difficult to interpret, we convert it to a percentile score. This score ranges from 0 to 100, and represents the percentage of law school matriculants with lower index scores. That is, a student with a percentile score of 75 has better academic credentials than three quarters of law school matriculants, but worse credentials than the remaining quarter. see http://public.econ.duke.edu/~hf14/teaching/povertydisc/readings/rothstein_yoon_2007.pdf

- first_pf - Whether or not the BAR exam was passed or failed
- region_first
- First year average grade (FYA) - Label used for training purposes. This is an Z score
Z-Scores are raw scores expressed in standard deviation units, relative to the mean score. Positive Z‐
scores indicate a raw score that is above the mean, negative Z‐scores indicate a raw score that is below
the mean, and a Z‐score of zero indicates a raw score that is equal to the mean. In a normally‐
distributed set of data, the general rule states that 68% of all scores will fall within ±1 SD of the mean;
95% of all scores will fall within ±2 SD, and 99.7% of all scores within ±3 SD. Z‐scores between ‐2.00 and
+2.00 are therefore considered relatively ordinary, while values greater than ‐2.00 and +2.00 are
unusual.

                      
In this scenario we will use the data to predict the success of prospective students in their first year(ZFYA) of law school. We can imagine such a model being used by an admissions department as part of deciding who to accept into their program. The school would also like to make sure these predictions are not biased by an individual’s race and sex. However, the LSAT, GPA, and FYA scores, may be biased due to social factors. 

we generate predictors Y for a logistic regression model, ranking the output. 

# About the Compas data

# About the Loan approval Data

https://datahack.analyticsvidhya.com/contest/practice-problem-loan-prediction-iii/
    

# Libraries

In [1]:
#pip install jupyter_contrib_nbextensions && jupyter contrib nbextension install 
#pip install autopep8
#jupyter nbextension enable --py widgetsnbextension
#python -m pip install git+https://github.com/dssg/aequitas.git


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

from io import StringIO
import pprint
import pandas_profiling
import re
import random
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display, clear_output, HTML
from aequitas.preprocessing import preprocess_input_df
import math
from collections import Counter
import heatmap
import gc
from heatmap import heatmap, corrplot
from ipywidgets import Layout, Button, Box
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.graph_objs import graph_objs as go 
import plotly.offline
from plotly.offline import iplot
import threading
import time
import ipyfilechooser
from ipyfilechooser import FileChooser
#$ pip install ipyupload
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from ipyupload import FileUpload
from itertools import cycle, chain, combinations

from IPython.core.display import display, HTML
display(HTML("<style>.container {width:80% !important;}</style>"))
%matplotlib inline

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Helper methods for both Classes

In [4]:
class helper_methods():

    def __init__(self):
        self.text_color = "#33C1FF"
        self.progressBar = widgets.FloatProgress(
            value=0.0, 
            min=0.0, 
            max=1.0,
            #layout = auto_width_layout,
        )
        
    
    def display_html(self, text, color, size):
        content = "<" + size + ">"  + "<text style='color:" + color + "'>" + text + "</text>" + "</" + size + ">" 
        display(HTML(content))
        
      
    def work_in_progress(self):
        total = 99 
        for i in range(total):
            if self.progressBar.value == 100:
                total = 99
            time.sleep(0.3)
            self.progressBar.value = float(i+1)/total

                
    #################################################################################################
    #  
    # 
    #################################################################################################
    def view_correlations(self, df, column_list, column_list_with_Y, protected_column_list, y_col ):
        
        feature_set_with_y = df
        out1 = widgets.Output(layout={})
        out2 = widgets.Output(layout={})
        out3 = widgets.Output(layout={})
        out4 = widgets.Output(layout={})
        tab_contents = [out1, out2, out3]
        
        children = tab_contents
        tab = widgets.Tab()
        tab.children = children
        
        tab.set_title(0, "Correlation plot")
        tab.set_title(1, "Pair Plot Features(X)")
        tab.set_title(2, "Pair Plot Features/y")
      
        display(out4)
        display(self.progressBar)
        display(tab)
        
        
        def f_cor(x ): #local method within view_correlations() funct
                plt.clf()
                plt.figure(figsize=(12, 12))           
                fig1 = corrplot(feature_set_with_y[column_list].corr(), size_scale=x, marker="o");       
                plt.show(fig1)
                plt.clf()
                plt.close(fig1)
                        
        
        def f_cor_val( a,b ): #local method within view_correlations() funct
                self.display_html ("The correlation value is " + str(feature_set_with_y[a].corr(feature_set_with_y[b])), self.text_color, "p")
        
        with out1:
            interact(f_cor, x = widgets.IntSlider(description = "slide to scale", min=0, max=70000, step=100, value=500, continuous_update=False), );
                    
        
            interact(f_cor_val, 
                        a = widgets.Dropdown(description = "Column 1", options = column_list_with_Y),
                        b = widgets.Dropdown(description = "Column 2", options = column_list_with_Y)
                );
        
                    
        def f_pp(_hue): #local method within view_correlations() funct
            self.progressBar.value = 0
            thread = threading.Thread(target=self.work_in_progress)
            thread.start()
            with out4:
                clear_output(wait = False)
                self.display_html("Calculating pair plot 1...", self.text_color, "p") 
            plt.figure(figsize=(6, 6))             
            sns.pairplot(feature_set_with_y, hue="race", diag_kind = 'kde', height=3, plot_kws = { 'alpha': 0.6,'s': 80, 'edgecolor': 'k'});  
            plt.show()
            plt.clf()
            plt.close()
            with out4:
                clear_output(wait = False)
                self.display_html("Calculating pair plot 2...", self.text_color, "p")
            with out3:
                clear_output(wait = False)
                sns.pairplot(feature_set_with_y,  hue="race", x_vars=column_list,y_vars=[y_col], height=10, aspect=0.7);
                plt.show()
                plt.clf()
                plt.close()
            with out4:
                clear_output(wait = False)
                self.display_html("Pair plots complete!", self.text_color, "p")
                self.progressBar.value = 100
            
        with out2:
            clear_output(wait = False)
            interact(f_pp, _hue = widgets.Dropdown(description = "Hue based on", 
                                                           options = protected_column_list));
            self.progress.value = 100
            
            
            
            
            
            
    #################################################################################################
    #  VIEW STATISTICS AROUND. THE PROTECTED FEATURES/ATTRIBUTES
    # 
    #################################################################################################        
            
    def view_protected(self, protected_attributes_list, y_value, data_frame, _w=600, _h=600, y_high_positive = True):
        protected = []
        protectedCount = []
        protectedPercentage = []
        HIGH_RANGE_POSITIVE = y_high_positive
        
        out1 = widgets.Output(layout={})
        out2 = widgets.Output(layout={})
        out3 = widgets.Output(layout={})
        out4 = widgets.Output(layout={})
        tab_contents = [out1, out2, out3]
        
        children = tab_contents
        tab = widgets.Tab(style={'description_layout':'auto', 'title_layout':'auto'})
        tab.children = children
        
        tab.set_title(0, "Group Representation in data")
        tab.set_title(1, "Outcome distribution")
        tab.set_title(2, "Group Positive outcome representation")
        
        local_layout = {'width': 'auto'}
        local_style = {'description_width':'initial'}
        display(out4)
        display(self.progressBar)
        display(tab)
        try:
            colType = data_frame[y_value].dtype
            if (colType != str and colType != object):
                self.progressBar.value = 0
                thread = threading.Thread(target=self.work_in_progress)
                thread.start()
                ####### view distribution of groups in the data
                with out1:
                    clear_output(wait = False)
                    self.plot_donut(protected_attributes_list,
                               data_frame,
                               w=_w, h=_h,
                               y_high_positive = HIGH_RANGE_POSITIVE,
                               title = "Representation of Protected group(s) in the data",
                               show_individual = False);
                
                    
                    def show_donut_for(choose): #local method within view_protected() funct
                        #plot the representation of data in the dataframe per protected group
                        with out4:
                            clear_output(wait = False)
                            print ("loading group representation in data...")
                        
                        thread = threading.Thread(target=self.work_in_progress)
                        thread.start()
                        self.plot_donut([choose],
                               data_frame,
                               w=_w, h=_h,
                               y_high_positive = HIGH_RANGE_POSITIVE,
                               title = "Representation of " + choose + " in the data",
                               show_individual = False);
                        self.progressBar.value = 100
                        with out4:
                            clear_output(wait = False)
                            print ("Finished loading")
                        
                
                    interact(show_donut_for, 
                        choose = widgets.Dropdown(description = "Select attribute", 
                                                  options = protected_attributes_list,
                                                 layout = local_layout,
                                                 style = local_style),
                            );
                
                
                ############## view distribution of results across groups
                with out2:
                    clear_output(wait = False)
                    #plot the distribution of Y-value per protected group
                    def show_distribution(choose, curve_type): #local method within view_protected() funct
                        #plot the representation of data in the dataframe per protected group
                        with out4:
                            clear_output(wait = False)
                            print ("loading outcome distribution...")
                        thread = threading.Thread(target=self.work_in_progress)
                        thread.start()
                        if choose != "--select--":
                            self.plot_distribution( [choose],
                                       y_value, 
                                       data_frame, 
                                       w=_w, h=_h, 
                                       y_high_positive = HIGH_RANGE_POSITIVE,
                                       curve_type = curve_type);
                        
                        self.progressBar.value = 100
                        with out4:
                            clear_output(wait = False)
                            print ("Finished loading")
                    interact(show_distribution, 
                            choose = widgets.Dropdown(description = "Select attribute",
                                                      options = ["--select--"] + protected_attributes_list,
                                                     layout = local_layout,
                                                     style = local_style),
                            curve_type = widgets.Dropdown(description = "Select Curve type", 
                                                          options = ["kde","normal"],
                                                         layout = local_layout,
                                                         style = local_style),
                            );

                with out4:#plot the representation of data in the dataframe per protected group
                    clear_output(wait = False)
                    print ("loading tab 3...")
                    
                    
                ############## view distribution of positive outcome across groups
                with out3:
                    clear_output(wait = False)
                    def show_outcome(choose, percentile_slider, top_x_slider): #local method within view_protected() funct
                        #plot the representation of data in the dataframe per protected group
                        with out4:
                            clear_output(wait = False)
                            print ("loading outcome across groups...")
                        thread = threading.Thread(target=self.work_in_progress)
                        thread.start()
                        if choose == "Mean":
                             #plot the distribution of above and below average score(y-value) per protected group
                            self.plot_outcome_representation(protected_attributes_list,
                                            y_value, data_frame,
                                            w=_w, h=_h,
                                            y_high_positive = HIGH_RANGE_POSITIVE,
                                              percentile = None,
                                              top_x = None)
                                 
                        if choose == "Percentile":
                        #plot the distribution of above and below average score(y-value) per protected group
                            self.plot_outcome_representation(protected_attributes_list,
                                            y_value, data_frame,
                                            w=_w, h=_h,
                                            y_high_positive = HIGH_RANGE_POSITIVE,
                                              percentile = percentile_slider,
                                              top_x = None)
                            
                        if choose == "Top-n":
                        #plot the distribution of above and below average score(y-value) per protected group
                            self.plot_outcome_representation(protected_attributes_list,
                                            y_value, data_frame,
                                            w=_w, h=_h,
                                            y_high_positive = HIGH_RANGE_POSITIVE,
                                              percentile = None,
                                              top_x = top_x_slider)

                    # see https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
                        
                        self.progressBar.value = 100
                        with out4:
                            clear_output(wait = False)
                            print ("Finished loading")
                                 
                    interact(show_outcome, 
                            choose = widgets.Dropdown(
                                description = "Select measure", 
                                options = ["Mean","Top-n","Percentile"],
                                layout = local_layout,
                                style = local_style),
                            percentile_slider = widgets.IntSlider(
                                description = "Select Percentile", 
                                min=0, max=100,
                                step=1, value=80, 
                                continuous_update=False,
                                layout = local_layout,
                                style = local_style), 
                            top_x_slider = widgets.IntSlider(
                                description = "Select n for Top_n", 
                                min=10, max=1000,
                                step=10, value=100, 
                                continuous_update=False,
                                layout = local_layout,
                                style = local_style),);

                    with out4:#plot the representation of data in the dataframe per protected group
                        clear_output(wait = False)
                        print ("Loaded...")
                    
                
                
                if (colType == str or colType == object):
                    self.display_html("Output label is not number and should be converted", self.text_color, "h4")
        except Exception as e:
            self.display_html("Something went wrong in view_protected method", self.text_color, "h4")
            print (e)
        gc.collect()
        self.progressBar.value = 100
          
            
            
    #################################################################################################
    #  
    # 
    #################################################################################################       
    def plot_donut(self, attributes_list, data_frame, w=800, h=800, y_high_positive = True, title = "Result",
                         show_individual = True):
    
        num_of_donuts = len(attributes_list)
        if num_of_donuts > 6:
            num_of_donuts = 6
            display (HTML("showing only the first 6 attributes"))
        
        sequential_color_list = [
            px.colors.sequential.Blues,
            px.colors.sequential.Greens, 
            px.colors.sequential.Oranges, 
            px.colors.sequential.Purples,
            px.colors.sequential.Reds,
            px.colors.sequential.Greys,
            px.colors.sequential.algae,
            px.colors.sequential.amp]
    
        color_pool = cycle(sequential_color_list)
    
        pie_list = []    
        a_l = []
        labels_arr = []
        values_arr = []
        color_arr = []
        annotations_arr = []
        annotate = dict(text='woops', 
            x=0.5, y=0.6, 
            font_size=15, 
            showarrow=False)
                     
        def create_label (row):#method local to this method
            
            names = list (row.index)
            values = list( row.values)
            text = ""
            for i in range (len(names)):
                text = text + ":" + names[i] + "_" + str(values[i])
            text = text[1:]            
            return text
            
        for a, pos in zip (attributes_list, range(len(attributes_list))):
            a_l.append(a)
            annotate['text'] = a
            annotate['y'] = annotate['y']-0.05
            annotations_arr.append(annotate.copy())
            #df = data_frame[a_l+[y]].groupby(a_l).count().reset_index().rename(columns={y: "values"})
            data_frame["count"] = 0
            df = data_frame[a_l+["count"]].groupby(a_l).count().reset_index().rename(columns={"count": "values"})
            df['labels'] =  df.apply(lambda row : create_label(row[a_l]), axis = 1) 
            #df['labels'] = df[a_l].astype(str).sum(axis=1) 
            df['values'].fillna(0,inplace=True)
            c = []
            s = []
            if pos == 0:
                for l in range (len(df['labels'].values)):
                    c.append(next(color_pool)[0])
                    if l >= len(sequential_color_list):
                        l = l - len(sequential_color_list)
                    s.append(l)
                df['colors'] = c
                df['color_pool_pos'] = s
                
            else:
                temp_list = list(df['values'].values)
                for count, color_index in zip(prev_counts, prev_color_pool) :
                    match = 0
                    for value, pos in zip (temp_list, range(len(temp_list))):
                        s.append(color_index)
                        try:
                            c.append (sequential_color_list[color_index][pos+1])
                        except:
                            c.append (sequential_color_list[color_index][2])
                        match = match + value
                        if match == count:
                            del temp_list[0:pos+1]
                            break
                df['colors'] = c
                df['color_pool_pos'] = s
            labels_arr.append (df['labels'])
            values_arr.append (df['values'])
            color_arr.append (df['colors'])
        
            prev_counts = df['values'].values
            prev_color_pool = df['color_pool_pos'].values

        
        hole = 0.8
        x1 = 0
        x2 =1
        y1 = 0
        y2 = 1
        adjust = round((1.0 - hole)* 0.5,2) 
        for x in range (num_of_donuts):
            pie_list.append(go.Pie(
            hole=hole, #Sets the fraction of the radius to cut out of the pie. Use this to make a donut chart
            sort=False,
            direction='clockwise',
            domain={'x': [x1, x2], 'y': [y1, y2]},
            values=values_arr[x],
            labels=labels_arr[x],
            #text = my_text,
            textinfo='label+percent',
            textposition='inside',
            name=attributes_list[x],
            marker={'colors': color_arr[x],'line': {'color': 'black', 'width': 1}}
            ))
            hole= round(hole - adjust, 2)
            x1 = round (x1 + adjust, 2)
            x2 = round (x2 - adjust, 2)
            y1 = round (y1 + adjust, 2)
            y2 = round (y2 - adjust, 2)
         
    
        fig = go.FigureWidget(data=pie_list)#need to reverse the order?
        fig.update_layout(autosize=False,
                      width=w,
                      height=h,
                      margin=dict(l=50,r=50,b=100, t=100,pad=4),
                      title=str(a_l),
                    # Add annotations in the center of the donut pies.
                    annotations=annotations_arr,
                    #paper_bgcolor='rgba(113, 136, 136, 1)', #for transparent set to (0,0,0,0)
                    #plot_bgcolor='rgba(113, 136, 136, 1)',
                    )

    
        fig.update_traces(textposition='inside')
        fig.update(layout_title_text=title,
               layout_showlegend=False)   
        
        fig.show()
        
        
         # # # # # Now create one donut per protected attribute for a clearer view if the call specifies this# # # # # # 
        if show_individual == True:
            specs = []
            fig2 = ""
            full_title =  ''
            if len(attributes_list) > 1:
                for x in range(len(attributes_list)):
                    specs.append ({'type':'domain'})
                fig2 = make_subplots(rows=1, cols=len(attributes_list), specs=[specs])
                for attribute, index in zip (attributes_list, range(len(attributes_list))):
                    full_title =  full_title + "/ " + attribute
                    df = data_frame[[attribute]+["count"]].groupby([attribute]).count().reset_index().rename(columns={"count": "values"})
                    pie = go.Pie(
                    hole=hole, #Sets the fraction of the radius to cut out of the pie. Use this to make a donut chart
                    sort=False,
                    direction='clockwise',
                    values=df["values"],
                    labels=df[attribute],
                    textinfo='label+percent',
                    textposition='inside',
                    name=attribute,
                    title = attribute,
                    #insidetextfont=dict(family="sans serif",size=18,color="crimson"),
            
                    marker={'colors': color_arr[x],'line': {'color': 'black', 'width': 1}} )
                    fig2.add_trace(pie, 1, index+1)
    
                fig2.update_layout(autosize=False,
                          width=600,
                          height=300,
                         #margin=dict(l=50,r=50,b=100, t=100,pad=4),
                  
                          )
    
    
                fig2.update_traces(textposition='inside')
                fig2.update(layout_title_text='Representation per attribute', layout_showlegend=True)   

                fig2.show()
        #as this can be a pointer to the input, clean it up
        data_frame.drop(["count"], axis=1, inplace = True)
        gc.collect()
        
        
        
        
    #################################################################################################
    #  
    # METHOD USED TO PLOT THE DISTRIBUTION OF THE OUTCOME ACROSS GROUPS
    #################################################################################################
    
    def plot_distribution(self, attributes_list, y, data_frame, w=800, h=800, y_high_positive = True, curve_type = "kde"):
        try:
            dist_arr = []
            g_labels = []
            for attribute, pos in zip (attributes_list, range(len(attributes_list))):
                groups = data_frame[attribute].unique()
                for value in range(len(groups)):
                    df = data_frame[ data_frame[attribute] == groups[value]]
                    dist_arr.append(df[y])
                    g_labels.append(attribute + "-" + str(groups[value]))
                # Add histogram data
                # Group data together
                hist_data = dist_arr
                group_labels = g_labels
                #Add the dist of all combined groups
                hist_data.append(data_frame[y])
                group_labels.append("All")
            
                # Create distplot with custom bin_size
                fig = ff.create_distplot(hist_data, group_labels, curve_type = curve_type) #, bin_size=[.1, .25, .5, 1]
                # Add title
                fig.update_layout(
                    autosize=False,
                    width=900,
                    height=500,
                    margin=dict(l=50,r=50,b=100, t=100,pad=4),
                    paper_bgcolor="LightSteelBlue",
                    title=y +' distribution across ' + attribute
                    )

                f2 = go.FigureWidget(fig)
                f2.show()
                #f = fig.show(renderer="notebook")
                groups = []
                dist_arr = []
                g_labels = []

        except Exception as e:
            self.display_html("Something went wrong generating the dist graph", self.text_color, "h4")
            print (e)
        gc.collect()
        
    
    #################################################################################################
    #  
    # 
    #################################################################################################
    def plot_outcome_representation(self, attributes_list, 
                                    y_value, data_frame, 
                                    w=800, h=800, 
                                    y_high_positive = True,
                                    percentile = None,
                                    top_x = None):
        
        num_of_attributes = len(attributes_list)
        if num_of_attributes > 6:
            num_of_attributes = 6
            display (HTML("showing only the first 6 attributes"))
        
        #create a new column that categorises positive outcome by default, as above average score and negative 
        #outcome as below average score
        y = y_value #Sort the naming out here later
        text = ""
        HIGH_RANGE_POSITIVE = y_high_positive
        yMin = round(data_frame[y_value].min(), 10)
        yDivPoint = data_frame[y_value].mean()
        yMax = data_frame[y_value].max()   
        
        outcome = ['', '']      
        if HIGH_RANGE_POSITIVE == True:
            # min to mean is neg, mean to max is pos
            outcome = ["Neg", "Pos"]
            #Rather than having defined bins of [yMin, yMean, yMax] base the display on precentiles.
            if not percentile == None:
                yDivPoint = data_frame[data_frame[y_value] >= np.percentile(data_frame[y_value],percentile)].min()[y_value]
            if not top_x == None:
                #Ascending means smallest to largest,
                print ("Top x =", data_frame.sort_values(y_value,ascending = True).head(top_x).shape[0])
                yDivPoint = data_frame.sort_values(y_value,ascending = False).head(top_x).min()[y_value]
                    
        if HIGH_RANGE_POSITIVE == False:
            outcome = ["Pos", "Neg"]
            # min to mean is pos, mean to max is neg
            #Rather than having defined bins of [yMin, yMean, yMax] base the display on precentiles.
            if not percentile == None:
                print ("bottom percentile = ", percentile)
                yDivPoint= data_frame[data_frame[y_value] <= np.percentile(data_frame[y_value],percentile)].max()[y_value]
            if not top_x == None:
                #Ascending means smallest to largest,
                print ("Top x =", data_frame.sort_values(y_value,ascending = True).head(top_x).shape[0])
                yDivPoint = data_frame.sort_values(y_value,ascending = True).head(top_x).max()[y_value]
        
        yMin = round(yMin, 3)
        yDivPoint = round(yDivPoint, 3)
        yMax = round(yMax, 3) 
        
        def get_next_prev_fraction(num):
            n = num
            if isinstance(n, float):
                s = str(n)
                _len = s[::-1].find('.')
                add = "0."
                if (_len > 0):
                    for x in range(_len-1):
                        add = add + "0"
                    add = add +"1"
                _next = round (n + float(add), _len)
                _prev = round (n - float(add), _len)
            return _next, _prev

        _, yDivPoint = get_next_prev_fraction(yDivPoint)
        bins = [yMin-1, yDivPoint, yMax+1]#adding + and -1 to account for any decimal issues
                    
                 
        
        impact_col_name = "impact"    

        data_frame[impact_col_name] = pd.cut(data_frame[y_value], bins, labels=outcome)
        

        x = "<br>* Output '" + str(y_value) + "' grouped in range:"
        text = text + "A high output is defined as a relevance(score) between " + str(yDivPoint) + " and " + str(yMax)+"<br>"
        text = text + "A low output is defined as a relevance(score) between " + str(yMin) + " and " + str(yDivPoint)
        display(HTML(text))
               
        if HIGH_RANGE_POSITIVE == True:
            impactTxt = "<b>Positive</b>"
        if HIGH_RANGE_POSITIVE == False:
            impactTxt = "<b>Negative</b>"
        x = "The Impact of a high output(ranking) on an individual or group is <b>" + impactTxt + "</b>"
        display(HTML(str(x)))
        

        sequential_color_list = [
            px.colors.sequential.Greens, 
            px.colors.sequential.Greys]

        
        self.plot_donut( list(attributes_list)+[impact_col_name], 
                                 data_frame, 
                                 w=600, h=600, 
                                 y_high_positive = True, 
                                 title = "Impact of outcome across group(s)", 
                                 show_individual = False)   
        #Here
        self.plot_donut( list(["race"])+[impact_col_name], 
                                 data_frame[(data_frame[impact_col_name]=="Pos")] ,
                                 w=600, h=600, 
                                 y_high_positive = True, 
                                 title = "Impact of outcome across feature..", 
                                 show_individual = False)   

        num_of_attributes = len(attributes_list)        
        def powerset(iterable):
            "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
            s = list(iterable)
            res = chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
            list_set = list(res)
            final_list = []
            for val in list_set:
                if len(val) == 2:
                    final_list.append(val)
            return final_list
        fig = go.Figure()
        if num_of_attributes > 1:
            comb = powerset(attributes_list)
            #for each combination of 2 protected attributes 
            for a, pos in zip (comb, range(len(comb))):
                df = data_frame[[a[0]]+[a[1]]+[impact_col_name]+[y_value]].groupby([a[0]]+[a[1]]+[impact_col_name]).count().reset_index().rename(columns={y_value: "values"})
                _total =  df['values'].sum()
                _total_pos =  df[(df[impact_col_name]=="Pos")]['values'].sum()
                _total_neg =  df[(df[impact_col_name]=="Neg")]['values'].sum()
                color_index = 7
                color_index_g = 7
                for value in df[a[1]].unique():
                    green = px.colors.sequential.Greens[color_index]
                    if color_index != 0:
                        color_index = color_index - 1
                    else:
                        color_index = 7
                    name = a[1]+"_"+str(value)
                    temp = df[(df[a[1]]==value) & (df[impact_col_name]=="Pos")]
                    display (temp)
                    _pcnt = list (  round((temp["values"]/_total) * 100 , 2)) 
                    _pcnt_from_total_pos = list ( round(  (temp["values"]/_total_pos) * 100, 2)) 
                    for x in range (len(_pcnt_from_total_pos)):
                        _pcnt_from_total_pos[x] = str(_pcnt_from_total_pos[x]) + "% of all Positive"
                    fig.add_trace(go.Bar(
                            y=df[a[0]].unique(),
                            x=temp["values"],
                            name='Positive-' + name,
                            text = _pcnt_from_total_pos,
                            textposition='auto',
                            orientation='h',
                            marker=dict(
                            color=green,
                            line=dict(color='rgba(58, 71, 80, 1.0)', width=2)
                        )
                    ))
                for value in df[a[1]].unique():
                    grey = px.colors.sequential.Oranges[color_index_g]
                    if color_index_g != 0:
                        color_index_g = color_index_g - 1
                    else:
                        color_index_g = 7
               
                    name = a[1]+"_"+str(value)
                    temp = df[(df[a[1]]==value) & (df[impact_col_name]=="Neg")]
                    _pcnt = list ( round(  (temp["values"]/_total) * 100 , 2) )
                    _pcnt_from_total_neg = list ( round(  (temp["values"]/_total_neg) * 100, 2) )
                    fig.add_trace(go.Bar(
                            y=df[a[0]].unique(),
                            x=temp["values"],
                            name='Negative_' + name,
                            text = _pcnt,
                            textposition='auto',
                            orientation='h',
                            marker=dict(
                            color=grey,
                            line=dict(color='rgba(58, 71, 80, 1.0)', width=2)
                        )
                    ))
                        

            fig.update_layout(title = "Impact of results on: "+ a[0]+ " and "+ a[1], 
                              barmode='stack',
                              xaxis_tickangle=-45,
                              autosize=False,
                              width=900,
                              height=h,
                              margin=dict(l=50,r=50,b=100, t=100,pad=4),
                             )
            fig.show()
            
            
        else:#only one protected group so just show it   
            df = data_frame[attributes_list+[impact_col_name]+[y_value]].groupby(attributes_list+[impact_col_name]).count().reset_index().rename(columns={y_value: "values"})
            temp = df[(df[impact_col_name]=="Pos")]
            _total =  df['values'].sum()
            _total_pos =  df[(df[impact_col_name]=="Pos")]['values'].sum()
            _total_neg =  df[(df[impact_col_name]=="Neg")]['values'].sum()
            fig.add_trace(go.Bar(
                            y=df[attributes_list[0]].unique(),
                            x=temp["values"],
                            name='Positive',
                            text = "add %",
                            textposition='auto',
                            orientation='h',
                            marker=dict(
                            color="green",
                            line=dict(color='rgba(58, 71, 80, 1.0)', width=2)
                        )
                    ))
            
            temp = df[(df[impact_col_name]=="Neg")]
            fig.add_trace(go.Bar(
                            y=df[attributes_list[0]].unique(),
                            x=temp["values"],
                            name='Negative',
                            text = "add %",
                            textposition='auto',
                            orientation='h',
                            marker=dict(
                            color="orange",
                            line=dict(color='rgba(58, 71, 80, 1.0)', width=2)
                        )
                    ))
            
            fig.update_layout(title = "Impact of results on: "+ str(attributes_list[0]), 
                              barmode='stack',
                              xaxis_tickangle=-45,
                              )
            fig.show()
        
            
        #as we added a column to this dataframe we will remove it here, as maybe a copy was not sent in
        data_frame.drop([impact_col_name], axis=1, inplace = True)
        gc.collect()

        
        
    def scale_normalise(self, attributes_list, data_frame, w=800, h=800, apply = False, s_type = "MIN_MAX_SCALAR"):
        
        print (s_type)
        scaler = preprocessing.StandardScaler()
        
        data_frame_to_scale = data_frame[attributes_list]
        
        if s_type == "STANDARD_SCALAR":
            scaler = preprocessing.StandardScaler()    
        elif s_type == "MIN_MAX_SCALAR":
            scaler = preprocessing.MinMaxScaler()
        elif s_type == "ROBUST_SCALAR":
            scaler = preprocessing.RobustScaler()
        elif s_type == "NORMALIZER":
            scaler = preprocessing.Normalizer()
         
        fig1, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 6))
        
        scaled_df = scaler.fit_transform(data_frame_to_scale)
        scaled_df = pd.DataFrame(scaled_df, columns=attributes_list)
        
        ax1.set_title('Before Scaling')
        for attribute in attributes_list:
            sns.kdeplot(data_frame[attribute], ax=ax1)
        
       
        ax2.set_title('After '+ s_type)
        for attribute in attributes_list:
            sns.kdeplot(scaled_df[attribute], ax=ax2)
        
        if apply == True:
            for attribute in attributes_list:
                data_frame[attribute] = scaled_df[attribute]
            return data_frame
        
        plt.show()

        plt.close(fig1)
        plt.clf()
        return None
    
    
    def label_encoding(self, attributes_list, data_frame):
        # creating initial dataframe
        labelencoder = LabelEncoder()
        return_dict = {}
        for attribute in attributes_list:
            categories = data_frame[attribute].unique()
            temp_df = pd.DataFrame(categories, columns=[attribute])
            # Assigning numerical values and storing in another column
            temp_df[attribute+"_cat"] = labelencoder.fit_transform(temp_df[attribute])
            # Convert this Temp_df into a dictionary
            temp_df.set_index(attribute+"_cat", inplace=True)
            return_dict.update(temp_df.to_dict())
            data_frame[attribute+"_cat"] = labelencoder.fit_transform(data_frame[attribute])
        return return_dict
    

In [7]:

#display ( dpUI.df.head())
helper_method = helper_methods()

#helper_method.view_protected( ["race","sex"], "ZFYA", dpUI.df[dpUI.df.ZFYA > np.percentile(dpUI.df.ZFYA,90)], _w=600, _h=600, y_high_positive = True)

# helper_method.plot_outcome_representation(["race","sex"], 
#                                         "ZFYA", 
#                                         dpUI.df, 
#                                         w=800, h=800, 
#                                         y_high_positive = True,
#                                         percentile = 90,
#                                         top_x = None)
        
        
helper_method.view_protected( ["race","sex"], "ZFYA", dpUI.df, _w=600, _h=600, y_high_positive = True)
 
# helper_method.scale_normalise(["LSAT", "sex"], 
#                     dpUI.df, 
#                     w=800, h=800, 
#                     apply = False, 
#                     s_type = "MIN_MAX_SCALAR");


# helper_method.label_encoding(["race", "sex"], 
#                      dpUI.df)




Output()

FloatProgress(value=0.0, max=1.0)

# Pre-process the data
Machine learning algorithms require numerical inputs. Our features will come in various forms. Some will be continuous, others categorical (in numeric or text format). Machine learning algorithms cannot work with variables in text form, we must perform certain preprocessing steps to get our data in the right format.


- Identify the protected attributes and the label (y) in the dataset so that these may be split  out when necessary
- Remove any obviously unnecessary columns
- Review any correlations and remove any obviously correlated input columns
- Standardise/normalise or scale any necessary numerical features
- Review protected attributes and
    * Consolidate values for better representation e.g Race => White and Non-White
    * Rename any necessary category/Text values (Label Encoding)
    * Change column data types of category/text data to 'int' (Label Encoding)
- one-hot encode the categorical features.
- Decide if the model will be trained with or without the protected attributes






In [5]:
class data_pre_process_UI():

    def __init__(self):
        self.helper_methods = helper_methods()
        pd.set_option('display.max_colwidth', -1)
        self.df = pd.DataFrame() # Original Dataframe uploaded
        self.df_url = None
        self.y_label_df = pd.DataFrame() # Dataframe containing label(score) and also the range(upper or lower)
        self.non_protected_feature_set_df = pd.DataFrame() # Dataframe containing features except protected
        self.protected_feature_set_df = pd.DataFrame() # Dataframe containing features except protected

        self.text_color = "#33C1FF"
        # A dictionary, mapping feature encoding values
        self.encoding_dict = {}
        self.HIGH_RANGE_POSITIVE = True
        self.renamed = {}
        self.referenceNames = {}

        
        self.style = {'description_width': 'initial'}
        self.layout = {'width': 'auto'}
        self.layout_short = {'width': 'auto'}
        
        self.auto_width_layout = {'width': 'auto'}     # override the default width of the button to 'auto' to let the button grow

        self.fileUploader = FileUpload(accept='.csv',
                                       multiple=False,
                                       disabled=False,
                                       style_button='color: darkblue; background-color: lightsalmon; width: 180px;',
                                       compress_level=9
                                       )
        

        self.missingDataRowsThresholdSlider = widgets.IntSlider(
                                value=70,
                                description= "Rows threshold(percent)" ,
                                max=100,
                                min=0,
                                style=self.style,
                                layout = self.auto_width_layout)
        
        
        self.missingDataColumnsThresholdSlider = widgets.IntSlider(
                                value=70,
                                description= "Columns threshold(percent)" ,
                                max=100,
                                min=0,
                                style=self.style,
                                layout = self.auto_width_layout)
        

        
        
        self.remove_rows_cols_button = widgets.Button(
            description='Remove',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Remove rows or columns where x% of data is missing',
            icon='',
            layout = self.auto_width_layout
        )
        
         
            
        self.selectColumnWithMissingData = widgets.Dropdown(
            options = ["--select--"],
            value = "--select--",
            description='Column with missing data:',
            disabled=False,
            style=self.style,   
            layout = self.auto_width_layout
        )
        
        
        self.selectImputeMethod = widgets.Dropdown(
            description='Replace missing values with:',
            disabled=False,
            style=self.style, 
            layout = self.auto_width_layout
        )
        
        
        self.apply_imputation_button = widgets.Button(
            description='Apply imputation',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Apply  imputation to the column',
            icon='',
            layout = self.auto_width_layout
        )
        
        
        self.selectYFeature = widgets.Dropdown(
            options=  self.df.columns.values,
            value=None,
            description='Output Label(y):',
            disabled=False,
            style=self.style,
            layout = self.auto_width_layout
        )
        

        self.selectXFeatures = widgets.SelectMultiple(
            options = self.df.columns.values,
            description='Features',
            disabled=False,
            style=self.style,
            layout = self.auto_width_layout
        )
        

        self.selectProtectedAttributes = widgets.SelectMultiple(
            options=self.df.columns.values,
            description='Protected Attribute(s)',
            disabled=False,
            style=self.style,
            layout = self.auto_width_layout
        )
        
        self.theSelectedInputFeaturesChoice = widgets.SelectMultiple(
            options = [],
            description='Features',
            disabled=False,
            style=self.style,
            layout = self.auto_width_layout
        )
        
        self.theSelectedInputFeaturesDropDown = widgets.Dropdown(
            options = [],
            description='Features',
            disabled=False,
            style=self.style,
            layout = self.auto_width_layout
        )
    

        self.selectImpact = widgets.RadioButtons(
            options={'A more positive impact on an individual': 1,
                     'A more negative impact on an individual': 0},
            description='A prediction in the high range will have:',
            disabled=False,
            style=self.style,
            layout = self.auto_width_layout
        )
        

        self.newNameForValue = widgets.Text(
            description='New Value Name',
            disabled=False,
            tooltip='enter the new value name',
            style = self.style,
            layout = self.auto_width_layout
        )
        

        self.choosenFeatureValues = widgets.SelectMultiple(
            description='Select value(s)',
            disabled=False,
            style=self.style,
            layout = self.auto_width_layout
        )


        self.merge_values_button = widgets.Button(
            description='Merge',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Merge feature values',
            icon='',
            layout = self.auto_width_layout
        )

        self.descriptionText = widgets.Text(
            description='Description of value',
            disabled=False,
            tooltip='Save a description for the value e.g "protected" or the original name',
            style = self.style,
            layout = self.auto_width_layout
        )

        self.save_description_button = widgets.Button(
            description='Save',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            icon='check',
            layout = self.auto_width_layout
        )


        self.label_encode_button = widgets.Button(
            description='Encode',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='click to apply label encoding',
            layout = self.auto_width_layout
        )
        

        self.one_hot_encode_button = widgets.Button(
            description='Encode',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='click to apply one-hot-encoding',
            layout = self.auto_width_layout
            
        )

        
        self.change_column_type_button = widgets.Button(
            description='Change',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            icon='check',
            layout = self.auto_width_layout
        )
        

        self.selectProtected = widgets.ToggleButtons(
            options=['Yes', 'No'],
            value='Yes',
            description='Select:',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltips=['Tooltip1', 'Tooltip2'],
            style=self.style,
            layout = self.auto_width_layout
        )

        
        self.selectColumnType = widgets.Dropdown(
            description='Change Column data type',
            options=['str', 'int'],
            value=None,
            disabled=False,
            style=self.style,
            layout = self.auto_width_layout
        )
        
        
        self.selectScaleType = widgets.Dropdown(
            description='Method to apply',
            options ={"--Select--":"SELECT",
                      "View Standard Scalar":"STANDARD_SCALAR",
                      "View Min Max Scalar": "MIN_MAX_SCALAR",
                      "View Robust Scalar":"ROBUST_SCALAR",
                      "View Normalise": "NORMALIZER"},
            value="SELECT",
            disabled=False,
            style=self.style,
            layout = self.auto_width_layout
        )

          
        self.view_attributes_button = widgets.Button(
            description='View',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            layout = self.auto_width_layout
            
        )
        
        
        self.apply_scale_button = widgets.Button(
            description='Apply',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            layout = self.auto_width_layout
            
        )
        
        self.view_correlation_button = widgets.Button(
            description='View',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='view correlations',
            layout = self.auto_width_layout
            
        )
        
        
        self.drop_features_button = widgets.Button(
            description='Drop',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Drop the selected features',
            layout = self.auto_width_layout
            
        )
        
        self.progressBar = widgets.FloatProgress(
            value=0.0, 
            min=0.0, 
            max=1.0,
            layout = self.auto_width_layout,
        )
        
        
        self.viewProtectedOut  = widgets.Output(layout={})        
#         self.results_accordion = widgets.Accordion(
#             children=[self.viewProtectedOut], 
#             titles=({0:'Results'}),
#             layout = self.auto_width_layout,
#             selected_index = None
#         )
        
        #functions called on buttons clicked....
        self.remove_rows_cols_button.on_click(self.on_remove_rows_cols_button_clicked)
        self.apply_imputation_button.on_click(self.on_apply_imputation_button_clicked)
        self.view_attributes_button.on_click(self.on_view_attributes_button_clicked)
        self.merge_values_button.on_click(self.on_merge_button_clicked)
        self.save_description_button.on_click(self.on_save_description_button_clicked)
        self.change_column_type_button.on_click(self.on_change_column_type_button_clicked)
        self.label_encode_button.on_click(self.on_label_encode_button_clicked)
        self.one_hot_encode_button.on_click(self.on_one_hot_encode_button_clicked)
        self.apply_scale_button.on_click(self.on_apply_scale_button_clicked)
        self.view_correlation_button.on_click(self.on_view_correlation_button_clicked)
        self.drop_features_button.on_click(self.on_drop_features_button_clicked)


        #interactive outputs, the functions will be called with any activity on the controls specified
        #the input to the function will also be the .value of the controls that cause invocation
        self.uploadFileOutput = widgets.interactive_output(self.uploadFileFunction, {
                                'fileUploader_value': self.fileUploader}) 
        self.uploadFileOutput.layout.height = '250px'
        
        self.dropOutput = widgets.interactive_output(self.dropImputeFunction, {
                        'missingDataColumnsThresholdSlider_value': self.missingDataColumnsThresholdSlider,
                        'missingDataRowsThresholdSlider_value': self.missingDataRowsThresholdSlider,
                        'selectColumnWithMissingData_value': self.selectColumnWithMissingData})
        self.imputeOutput = widgets.Output(layout={})
        self.selectLabelYOutput = widgets.interactive_output(self.selectLabelYFunction, {
                                'selectYFeature_values': self.selectYFeature})
        self.selectLabelYOutput.layout.height = '100px'
        self.selectLabelXOutput = widgets.interactive_output(self.selectLabelXFunction, {
                                'selectXFeatures_values': self.selectXFeatures,
                                'selectProtectedAttributes_values': self.selectProtectedAttributes})
        self.selectLabelXOutput.layout.height = '150px'
        self.selecImpactOutput = widgets.interactive_output(self.selectImpactFunction, {
                                'selectImpact_values': self.selectImpact})
        
        self.workInProgressOut = widgets.Output(layout={})
        self.mergeOut = widgets.interactive_output(self.merge_Feature, {
                        'theSelectedInputFeaturesDropDown_value': self.theSelectedInputFeaturesDropDown,})
        self.saveDescriptionOut  = widgets.Output(layout={})
        

        
        self.theSelectedInputFeaturesChoiceOut = widgets.interactive_output(self.selected_features_choice, {
                        'theSelectedInputFeaturesChoice_value': self.theSelectedInputFeaturesChoice,})
        self.theSelectedInputFeaturesChoiceOut.layout.height = '100px'
        
        self.theSelectedInputFeaturesDropDownOut = widgets.interactive_output(self.selected_features_drop_down, {
                        'theSelectedInputFeaturesDropDown_value': self.theSelectedInputFeaturesDropDown,})
        self.theSelectedInputFeaturesDropDownOut.layout.height = '50px'
        
        self.LEOut = widgets.Output(layout={})

        self.HotEncodeOut = widgets.Output(layout={})

        self.scaleNormaliseOut = widgets.interactive_output(self.on_select_scale_type, {
                        'selectScaleType_value': self.selectScaleType,})
        self.changeDataTypeOut = widgets.Output(layout={})

        self.correlationsOut = widgets.Output(layout={})
        self.correlationsGraphOut = widgets.Output(layout={})
        self.dropColOut = widgets.Output(layout={})           

        self.protectedOut = widgets.interactive_output(self.withOrWithoutProtected, {
                            'selectProtected_value': self.selectProtected})
       
    
        #Create and display a FileChooser widget with path = current path
        self.fc = FileChooser()
        self.fc.show_hidden = True
        # Show or hide folder icons
        self.fc.use_dir_icons = True
        self.fc.default_filename = 'data.csv'
        self.fc._select.description = "Save File as.."

        


            
    #################################################################################################
    #  
    # 
    #################################################################################################     
    def uploadFileFunction(self, fileUploader_value):
        try:
            for key in fileUploader_value:
                text = "File uploaded: " + fileUploader_value[key]['metadata']['name']
                self.df_url = fileUploader_value[key]['metadata']['name']
                self.helper_methods.display_html (text, self.text_color, "h4")
                try:
                    csvInBytes = fileUploader_value[key]['content']
                    s = str(csvInBytes, 'utf-8')
                    data = StringIO(s)
                    self.df = pd.read_csv(data)
                    if self.df.columns[0] ==  "Unnamed: 0":
                        self.df = self.df.drop(["Unnamed: 0"], axis=1)
                    del csvInBytes
                    del data
                    del s
                    self.helper_methods.display_html ("Sample of data",self.text_color, "h3")
                    display (self.df.head(5))
                    self.selectColumnWithMissingData.options = self.df.columns[self.df.isnull().any()]
                    
                        
                except BaseException as error:      
                    text = "Error reading file - upload new file"
                    self.helper_methods.display_html (text, "red")
                    display('An exception occurred: {}'.format(error))
        except BaseException as error:
            display("Error uploading file - upload again")
            display('An exception occurred: {}'.format(error))
            
        self.selectXFeatures.options = self.df.columns.values
        self.selectProtectedAttributes.options = self.df.columns.values
        self.selectYFeature.options = self.df.columns.values
        gc.collect()

    #################################################################################################
    #  
    # 
    ################################################################################################# 
    def on_remove_rows_cols_button_clicked(self, b):
        thresholdCol = self.missingDataColumnsThresholdSlider.value
        thresholdRow = self.missingDataRowsThresholdSlider.value
        #Dropping columns with missing value rate higher than threshold
        self.df = self.df[self.df.columns[self.df.isnull().mean() < thresholdCol]]
            
        #Dropping rows with missing value rate higher than threshold
        self.df = self.df.loc[self.df.isnull().mean(axis=1) < thresholdRow]
        self.on_display_missing_data_info()

    
    #################################################################################################
    #  
    # 
    ################################################################################################# 
    def on_apply_imputation_button_clicked(self, b):
        #Filling all missing values with 0
        self.df[self.selectColumnWithMissingData.value].fillna(self.selectImputeMethod.value, inplace=True)
        self.on_display_missing_data_info()
        
    #################################################################################################
    #  
    # 
    ################################################################################################# 
    def on_display_missing_data_info(self):
        with self.dropOutput:
            clear_output(wait=True)
            thresholdCol = self.missingDataColumnsThresholdSlider.value
            thresholdRow = self.missingDataRowsThresholdSlider.value

            null_columns=self.df.columns[self.df.isnull().any()]
            all_columns=self.df.columns.values
            if len(null_columns) != 0:
                #How many columns have missing value rate higher than threshold
                text = "Number of columns with missing data ratio above " + str(thresholdCol) + " percent"
                num = self.df[self.df.columns[self.df.isnull().mean() > thresholdCol]].any(axis=0).sum()
                text = text + " is: " + str(num)
                self.helper_methods.display_html (text, self.text_color, "p")
            
                #How many rows with missing value rate higher than threshold
                text = "Number of rows with missing data ratio above "  + str(thresholdRow) + " percent"
                num = self.df.loc[self.df.isnull().mean(axis=1) > thresholdRow].any(axis=1).sum() 
                text = text + " is: " + str(num)
                self.helper_methods.display_html (text, self.text_color, "p")
            
                num_rows = self.df.shape[0] 
                text = "Total Number of Rows: "+ str(num_rows)
                self.helper_methods.display_html (text, self.text_color, "p")
            
                num_cols = self.df.shape[1] 
                text = "Total Number of Columns: "+ str(num_cols)
                self.helper_methods.display_html (text, self.text_color, "p")
            
                num_rows = self.df.isnull().any(axis=1).sum() 
                text = "Number of Rows with missing data: "+ str(num_rows)
                self.helper_methods.display_html (text, self.text_color, "p")
                #display(self.df[self.df.isnull().any(axis=1)][all_columns].head())
            
                self.selectColumnWithMissingData.options = self.df[null_columns].isnull().sum().index
                num_cols = len(self.selectColumnWithMissingData.options)
                text = "Number of Columns with missing data:" + str(num_cols)
                self.helper_methods.display_html (text, self.text_color, "p")
            else:
                num_rows = self.df.shape[0] 
                text = "Total Number of Rows: "+ str(num_rows)
                self.helper_methods.display_html (text, self.text_color, "p")
            
                num_cols = self.df.shape[1] 
                text = "Total Number of Columns: "+ str(num_cols)
                self.helper_methods.display_html (text, self.text_color, "p")
                
                text = "No missing data detected"
                self.helper_methods.display_html (text, "green", "p")
            return null_columns
        
        
    #################################################################################################
    #  
    # 
    #################################################################################################    
    def dropImputeFunction (self, missingDataColumnsThresholdSlider_value, missingDataRowsThresholdSlider_value, selectColumnWithMissingData_value):
        if not self.df.empty:
            null_columns = self.on_display_missing_data_info()
            
            numeric = self.df[null_columns].median().index
            with self.imputeOutput:
                clear_output(wait=True)
                optionsDic = {}
                if len(null_columns) != 0:
                    if selectColumnWithMissingData_value in numeric:
                    
                        median = self.df[selectColumnWithMissingData_value].median()
                        text = "The Median value for column: "+ selectColumnWithMissingData_value + " is " + str(median)
                        optionsDic["Median"] = median
                        optionsDic["Zero"] = 0
                        self.selectImputeMethod.options = optionsDic
                        self.helper_methods.display_html (text, self.text_color, "p")
                    
                    if not selectColumnWithMissingData_value in numeric:
                        display(self.df[selectColumnWithMissingData_value].value_counts().to_frame())
                        most_frequent = self.df[selectColumnWithMissingData_value].value_counts().idxmax()                
                        text = "Most frequent categorical value for " + selectColumnWithMissingData_value + " is " + str(most_frequent)
                        optionsDic["Most Frequent"] = str(most_frequent)
                        optionsDic["Other"] = "Other"
                        self.selectImputeMethod.options = optionsDic
                        self.helper_methods.display_html (text, self.text_color, "p")
                
                    num_missing = self.df[selectColumnWithMissingData_value].isnull().sum()
                    text = "Instances of missing data for selected column is " + str(num_missing)
                    self.helper_methods.display_html (text, self.text_color, "p")
                else:
                    text = "No missing data detected"
                    self.helper_methods.display_html (text, "green", "p")
    
                    
    #################################################################################################
    #  
    # 
    #################################################################################################                 
    def selectLabelYFunction(self, selectYFeature_values):
        
        if not self.df.empty:
           
            #Once we select a label, remove it from possible input features dropdown
            try:
                #store the entire label column in its own dataframe
                self.y_label_df = self.df[selectYFeature_values].copy().to_frame()
           
                listx = list(self.df.columns.values) 
                #use different ways to remove an item of the list
                listx.remove(selectYFeature_values) 
                #converting the tuple to list
                tuplex = tuple(listx)
                self.selectXFeatures.options =tuplex
                self.selectProtectedAttributes.options =tuplex

            
            except:
                self.selectXFeatures.options = self.df.columns.values
                self.selectProtectedAttributes.options = self.df.columns.values
                
        
            if selectYFeature_values == None:
                self.helper_methods.display_html ("None Selected", self.text_color, "h5")
            else:
                self.helper_methods.display_html (str(selectYFeature_values), self.text_color, "h5")
        
        
            if selectYFeature_values != None:
                colType = self.df[selectYFeature_values].dtype
                if (colType != str and colType != object):
                    yMin = round(
                        self.df[selectYFeature_values].min(), 3)
                    yMean = round(
                        self.df[selectYFeature_values].mean(), 3)
                    yMax = round(
                        self.df[selectYFeature_values].max(), 3)
                
                    text = "Low Range(Minimum to Mean value): = " + str(yMin) + " to " + str(yMean) + "<br>"
                    text = text + "High Range(Mean to Maximum value): = " + str(yMean) + " to " + str(yMax) + "<br>"
                    self.helper_methods.display_html (text, self.text_color, "p")
                 
                if (colType == str or colType == object):
                    self.helper_methods.display_html ("!The selected output label is not numeric", "orange", "p")
    
            
            
    #################################################################################################
    #  
    # 
    #################################################################################################           
    def selectImpactFunction(self, selectImpact_values):
        
        #Default value is High
        self.helper_methods.display_html ("Your choice indicates that the higher the predicted score the better the outcome for the individual", self.text_color,  "p")

        if not self.y_label_df.empty and not self.protected_feature_set_df.empty:
            if selectImpact_values == 1:
                self.HIGH_RANGE_POSITIVE = True
                self.helper_methods.display_html ("Your choice indicates that the higher the predicted score the better the outcome for the individual", self.text_color,  "p")
            else:
                self.HIGH_RANGE_POSITIVE = False
                self.helper_methods.display_html ("Your choice indicates that the lower the predicted score the better the outcome for the individual",  self.text_color, "p")

            with self.viewProtectedOut:
                clear_output(wait=True)
    
    
    #################################################################################################
    #  
    # 
    ################################################################################################# 
    def selectLabelXFunction(self, selectXFeatures_values, selectProtectedAttributes_values):
        if not self.df.empty:
            self.non_protected_feature_set_df = self.df
            #drop list contains all the Features not yet selected as X value.
            drop_list = set(selectXFeatures_values).symmetric_difference(
                self.selectXFeatures.options)
            self.helper_methods.display_html ("- Possible input features(X):","black", "h4")
            if len(selectXFeatures_values) == 0:
                self.helper_methods.display_html ("None Selected", self.text_color, "h4")
            else:
                self.helper_methods.display_html (str(selectXFeatures_values), self.text_color, "h4")
        
            self.helper_methods.display_html ("- Feature identifying membership of a protected group:", "black", "h4")
            if len(selectProtectedAttributes_values ) == 0:
                self.helper_methods.display_html ("None Selected", self.text_color, "h5")
            else:
                self.helper_methods.display_html (str(selectProtectedAttributes_values ), self.text_color, "h4")
            
            for c in drop_list:
                try:
                    self.non_protected_feature_set_df = self.df.drop(columns = c, axis=1)
                except:
                    print(c + "not a selected feature")
            self.non_protected_feature_set_df = self.non_protected_feature_set_df.drop(columns = self.selectYFeature.value, axis=1) 
        
            
            if len(selectProtectedAttributes_values) != 0:
                for c in list(selectProtectedAttributes_values):
                    try:
                        self.non_protected_feature_set_df = self.non_protected_feature_set_df.drop(columns = c, axis=1) 
                    except:
                        print(c + "not a selected feature")
                self.protected_feature_set_df = self.df[list(selectProtectedAttributes_values)].copy()

            #get a list of protected and non-protected based on the data frame 
            self.get_all_engineered_features()[2]#we only want 3d output of funct
            
    #################################################################################################
    #  get_dataframe()[0] to only return df
    # get_dataframe()[1] to only return column values
    ################################################################################################# 
    def get_dataframe(self, include_label_y, include_protected):
        df =  self.non_protected_feature_set_df
        if include_protected == True:
            df = pd.concat([df, self.protected_feature_set_df], axis=1)   
        if include_label_y == True:
            df = pd.concat([df, self.y_label_df], axis=1)
        return df, df.columns.values
     
        
     
    
    def work_in_progress(self, progress):
        total = 100
        for i in range(total):
            time.sleep(0.2)
            progress.value = float(i+1)/total
            
    

    #################################################################################################
    #  
    # 
    #################################################################################################
    def on_view_attributes_button_clicked (self, b):
        if not self.y_label_df.empty and not self.protected_feature_set_df.empty:
            feature_set_with_y = self.get_dataframe(include_label_y = True, include_protected = True)[0]
            thread = threading.Thread(target=self.work_in_progress)
            thread.start()
            try:
                with self.viewProtectedOut:
                    clear_output(wait=True)
                    self.helper_methods.view_protected(self.selectProtectedAttributes.value, 
                                                       self.selectYFeature.value, feature_set_with_y, 
                                                       600, 600, self.HIGH_RANGE_POSITIVE)
                    
            except:
                print("an error occured viewing protected group")
        
            self.progressBar.value = 100
    
    
    #################################################################################################
    #  
    # 
    #################################################################################################
    def get_all_engineered_features(self): 
        protected = []
        non_protected = []
        all_features = []
        for feature in self.protected_feature_set_df.columns.values:
            protected.append(feature)
            all_features.append(feature)
            
        for feature in self.non_protected_feature_set_df.columns.values:
            non_protected.append(feature)
            all_features.append(feature)
        self.theSelectedInputFeaturesDropDown.options = all_features 
        self.theSelectedInputFeaturesChoice.options = all_features 
        return protected, non_protected, all_features
        
    #################################################################################################
    #  
    # 
    #################################################################################################
    def merge_Feature(self, theSelectedInputFeaturesDropDown_value):
        if not self.non_protected_feature_set_df.empty:

            if not self.y_label_df.empty and not self.non_protected_feature_set_df.empty:
                try:

                    protected, non_protected, all_features = self.get_all_engineered_features()
           
                    p = set(protected)# for fast reference operation
                    
                    
                    if theSelectedInputFeaturesDropDown_value in p:# if protected
                        self.choosenFeatureValues.options = self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].unique()
                        colType = self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].dtype
                    else: 
                        self.choosenFeatureValues.options = self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].unique()
                        colType = self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].dtype
                    if len(self.choosenFeatureValues.value) == 1:
                        self.merge_values_button.description = "Change value"
                    else:
                        self.merge_values_button.description = "Merge values"
                    with self.mergeOut:
                        clear_output(wait=True)
                        if theSelectedInputFeaturesDropDown_value in p:
                            text = "The selected Feature is one of your protected Features."
                            self.helper_methods.display_html (text, self.text_color, "p")
                            
                    with self.changeDataTypeOut:
                        clear_output(wait=True)
                        text = "Feature data type: " + str(colType)
                        self.helper_methods.display_html (text, self.text_color, "h4")
                        
                except Exception as e:
                    
                    self.helper_methods.display_html ("**Protected attribut merge not yet generated**", self.text_color, "h4")
                    print(e)
    
    
    
    #################################################################################################
    #  
    # 
    #################################################################################################
    def on_merge_button_clicked(self, b):
        with self.viewProtectedOut:
            clear_output(wait=True)
            display (HTML("Click again to view with newly merged values"))
        with self.mergeOut:
            clear_output(wait=True)
            protected = []
            for feature in self.protected_feature_set_df.columns.values:
                protected.append(feature)
            p = set(protected)# for fast reference operation, to see if we are renaming a protected Feature.
            
            colTypeCast = type(self.choosenFeatureValues.value[0])
            
            if self.theSelectedInputFeaturesDropDown.value not in self.renamed:
                self.renamed[self.theSelectedInputFeaturesDropDown.value] = {}
                
            for value in self.choosenFeatureValues.value:
                newValue = colTypeCast(self.newNameForValue.value)
                self.renamed[self.theSelectedInputFeaturesDropDown.value][value] = newValue
                newValue = colTypeCast(self.newNameForValue.value)
                
                if self.theSelectedInputFeaturesDropDown.value in p:# if protected
                    if not self.theSelectedInputFeaturesDropDown.value+"_bm" in self.protected_feature_set_df.columns:
                        self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value+"_bm"] = self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value]
                    self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].replace(
                        [value], newValue, inplace=True)
                else:
                    if not self.theSelectedInputFeaturesDropDown.value+"_bm" in self.non_protected_feature_set_df.columns:
                        self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value+"_bm"] = self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value]
                    self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].replace(
                        [value], newValue, inplace=True) #######
     
            ####show stats after rename
            display("Statistics after Merge:")
            if self.theSelectedInputFeaturesDropDown.value in p:# if protected
                display(self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].value_counts())
                colType = self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].dtype
                self.choosenFeatureValues.options = self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].unique()
           
            else:
                display(self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].value_counts())
                colType = self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].dtype
                self.choosenFeatureValues.options = self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].unique()
           
            if colType == str or colType == object :
                colType = str
                self.choosenFeatureValues.value = tuple([colType(newValue)])
                
            if colType == int or colType == float:
                self.choosenFeatureValues.value = tuple([newValue,])
                
            self.newNameForValue.value = "" #Reset the new name text box
                           
                        
    #################################################################################################
    #  
    # 
    #################################################################################################        
    def on_save_description_button_clicked(self, b):
        with self.saveDescriptionOut:
            clear_output(wait=True)
            
            if self.theSelectedInputFeaturesDropDown.value not in self.referenceNames:
                    self.referenceNames[self.theSelectedInputFeaturesDropDown.value] = {}
            #print ("options selected", self.theSelectedInputFeaturesDropDown.value)
            for value in self.choosenFeatureValues.value:
            
                text = "Saving description of: " + str(value) + " as "+ str(self.descriptionText.value) 
                display(text)
                self.referenceNames[self.theSelectedInputFeaturesDropDown.value][value] = self.descriptionText.value
                print(self.referenceNames)
        self.descriptionText.value = ""
    
   
    #################################################################################################
    #  displayed when values selected for hot-encoding
    # 
    #################################################################################################
    def selected_features_drop_down(self, theSelectedInputFeaturesDropDown_value):
        if not self.df.empty:
            with self.theSelectedInputFeaturesDropDownOut:
                clear_output(wait=True)
                protected, non_protected, all_features = self.get_all_engineered_features()
                selected = theSelectedInputFeaturesDropDown_value
                text = selected
                if selected in protected:# if protected
                    text = text + ": values " + str(self.protected_feature_set_df[selected].unique())
                    self.helper_methods.display_html (text, self.text_color, "h4")
                else: 
                    text = text + ": values " + str(self.non_protected_feature_set_df[selected].unique())
                    self.helper_methods.display_html (text, self.text_color, "h4")
                    
                        
    #################################################################################################
    # called when the theSelectedInputFeaturesChoice_value changes(e.g when selected for one-hot encode, 
    # scaling, etc
    #################################################################################################
    def selected_features_choice(self, theSelectedInputFeaturesChoice_value):
        if not self.df.empty:
            with self.theSelectedInputFeaturesChoiceOut:
                clear_output(wait=True)
                protected, non_protected, all_features = self.get_all_engineered_features()
                #Show th values when one feature is selected, just for convenience before selecting
                if len (theSelectedInputFeaturesChoice_value) == 1:
                    for selected in theSelectedInputFeaturesChoice_value:
                        text = selected
                        if selected in protected:# if protected
                            text = text + ": values to encode " + str(self.protected_feature_set_df[selected].unique())
                            self.helper_methods.display_html (text, self.text_color, "h4")
                        else: 
                            text = text + ": values to encode " + str(self.non_protected_feature_set_df[selected].unique())
                            self.helper_methods.display_html (text, self.text_color, "h4")

    #################################################################################################
    #  
    # 
    #################################################################################################                           
    def on_label_encode_button_clicked(self, b):
         if not self.non_protected_feature_set_df.empty:
            try:
                protected, non_protected, all_features = self.get_all_engineered_features()
                p = set(protected)# for fast reference operation
                prot_list = []
                non_prot_list = []
                prot_dict = {}
                non_prot_dict = {}
                for selected in self.theSelectedInputFeaturesChoice.value:
                    if selected in p:# if protected
                        prot_list.append(selected)   
                    else: 
                        non_prot_list.append(selected)

                if len(prot_list) > 0:
                    prot_dict = self.helper_methods.label_encoding(prot_list, self.protected_feature_set_df)                  
                    self.encoding_dict.update(prot_dict)
                if len(non_prot_list) > 0:
                    non_prot_dict = self.helper_methods.label_encoding(non_prot_list, self.non_protected_feature_set_df)                  
                    self.encoding_dict.update(non_prot_dict)
                
                text = ""
                for v in prot_list:
                    text = text + v + "_cat" + ", "
                for v in non_prot_list:
                    text = text + v + "_cat" + ", "
                with self.LEOut:
                    clear_output(wait=True)
                    text = "Label Encoding applied, new columns are  " +  text
                    self.helper_methods.display_html (text, self.text_color, "p")
                    self.helper_methods.display_html (str(self.encoding_dict), self.text_color, "p")
                    self.get_all_engineered_features()  #to refresh the dropdowns with the new cols      
                    
            except Exception as e:
                with self.LEOut:
                    clear_output(wait=True)
                    self.helper_methods.display_html ("**Error when label encoding**", self.text_color, "h4")
                    display(e)
            
        
    #################################################################################################
    #  
    # 
    ################################################################################################# 
    def on_one_hot_encode_button_clicked(self, b):
        """ One Hot Encoding is used to convert categorical data, or text data, into numbers, 
        which our predictive models can better understand. 
        label encoding (an alternative approach)might introduces a new problems e.g if
        we encoded a set of country names into numerical data. 
        This is actually categorical data and there is no relation, of any kind, between the rows.
        The problem here is, since there are different numbers in the same column, 
        the model will misunderstand the data to be in some kind of order, 0 < 1 < 2. 
        But this isn’t the case at all. To overcome this problem, we use One Hot Encoder.
        One hot encoding is a process by which categorical variables are converted into a form that could be
        provided to ML algorithms to do a better job in prediction. 
        The categorical values start from 0 goes all the way up to N-1 categories.
        What one hot encoding does is, it takes a column which has categorical data, 
        which has been label encoded, and then splits the column into multiple columns. 
        The numbers are replaced by 1s and 0s, depending on which column has what value. 
        We want 7 new columns one for each Race category in our dataset."""
        with self.HotEncodeOut:
            clear_output(wait=True)
            protected, non_protected, all_features = self.get_all_engineered_features()
            p = set(protected)# for fast reference operation
                
            selected = self.theSelectedInputFeaturesDropDown.value
            print("Applying one hot encoding to feature", selected)
            if selected in p:# if protected
                self.protected_feature_set_df[selected] = pd.Categorical(self.protected_feature_set_df[selected])
                dfDummies = pd.get_dummies(self.protected_feature_set_df[selected], prefix=selected)
                self.protected_feature_set_df = pd.concat([self.protected_feature_set_df, dfDummies], axis=1)
                self.helper_methods.display_html("Generated Features: "+  str(dfDummies.columns.values), self.text_color, "h5")
            else: 
                self.non_protected_feature_set_df[selected] = pd.Categorical(self.non_protected_feature_set_df[selected])
                dfDummies = pd.get_dummies(self.non_protected_feature_set_df[selected], prefix=selected)
                self.non_protected_feature_set_df = pd.concat([self.non_protected_feature_set_df, dfDummies], axis=1)
                self.helper_methods.display_html("Generated Features: "+ str(dfDummies.columns.values), self.text_color, "h5")

            self.get_all_engineered_features()  #to refresh the dropdowns with the new cols

    #################################################################################################
    #  
    # 
    ################################################################################################# 
    def on_change_column_type_button_clicked(self, b):
        with self.changeDataTypeOut:
            clear_output(wait=True)
            protected = []
            for feature in self.protected_feature_set_df.columns.values:
                protected.append(feature)
            p = set(protected)# for fast reference operation, to see if we are renaming a protected Feature.
            
            colName = self.theSelectedInputFeaturesDropDown.value
            newColType = self.selectColumnType.value
            
            
            if self.theSelectedInputFeaturesDropDown.value in p:# if protected
                colType = self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].dtype
                text_orig = "Original Data Type:" + str(colType)
                self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value] = self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].astype(newColType) 
                colType = self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].dtype
            
            else:
                colType = self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].dtype
                text_orig = "Original Data Type:" + str(colType)
                self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value] = self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].astype(newColType) 
                colType = self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].dtype
            
            self.helper_methods.display_html (text_orig, self.text_color, "p")
            text_new = "New Data Type:" + str(colType)
            self.helper_methods.display_html (text_new, self.text_color, "p")
            #self.refresh_views("change_column_type")
    
    
    
    #################################################################################################
    #  Method to view the scale of the input features. on_apply_scale button calls the same method
    # but with apply set to True for the helper file
    #################################################################################################
    def on_select_scale_type(self, selectScaleType_value):
        if not self.df.empty:
            if selectScaleType_value != "SELECT":
                self.scale_features(_apply = False)     
            
    def on_apply_scale_button_clicked(self, b):
        if not self.df.empty:
            self.scale_features(_apply = True)  
            
    #################################################################################################
    #  HERE: Have to view when select value, and apply when click button(so need to modify this flow)
    # 
    #################################################################################################
    def scale_features(self, _apply = False):
        with self.scaleNormaliseOut:
            clear_output(wait=True)
            try:
                protected, non_protected, all_features = self.get_all_engineered_features()
                p = set(protected)# for fast reference operation
                prot_list = []
                non_prot_list = []
                all_list = []

                for selected in self.theSelectedInputFeaturesChoice.value:
                    if selected in p:# if protected
                        prot_list.append(selected) 
                        all_list.append(selected) 
                    else: 
                        non_prot_list.append(selected)
                        all_list.append(selected)
                
                returned_df = self.helper_methods.scale_normalise(all_list, 
                                                    self.get_dataframe( include_label_y = False, include_protected = True)[0],                      
                                                    w=12, h=6, 
                                                    apply = _apply, 
                                                    s_type = self.selectScaleType.value);
                
                if _apply == True:
                    for col in prot_list:
                        self.protected_feature_set_df[col] = returned_df[col];
                    for col in non_prot_list:
                        self.non_protected_feature_set_df[col] = returned_df[col];
                    self.helper_methods.display_html ("Operation has been applied", self.text_color, "p")

                else:
                    self.helper_methods.display_html ("View only, press 'Apply' to apply", self.text_color, "p")
                
                
                del returned_df
                gc.collect()
                            
            except Exception as e:
                    self.helper_methods.display_html ("**Error when applying scale, select only numeric features**", self.text_color, "h4")
                    print(e)
            
        
    
    #################################################################################################
    # view_correlations: visualize the relationship between the features and the response using scatterplots
    # we need to choose variables that we think will be good predictors for the dependent variable — that can
    # be done by checking the correlation(s) between variables, by plotting the data and searching visually
    # for relationship, by conducting preliminary research on what variables are good predictors of y
    #################################################################################################
    def on_view_correlation_button_clicked(self, b):
        if not self.df.empty:
            try:
                with self.correlationsGraphOut :
                    clear_output(wait=False)
                    feature_set_with_y = self.get_dataframe(include_label_y = True, include_protected = True)[0]
                    column_list = list(self.theSelectedInputFeaturesChoice.value)
                    column_list_with_Y = list(self.theSelectedInputFeaturesChoice.value)+[self.selectYFeature.value]
                    protected_column_list = self.protected_feature_set_df.columns
                    self.helper_methods.view_correlations(feature_set_with_y, 
                                                          column_list, 
                                                          column_list_with_Y, 
                                                          protected_column_list,
                                                         self.selectYFeature.value)
            except:
                print("No values selected yet")
                
                
    #################################################################################################
    # Drop any columns that will not be used as input feature 
    # 
    #################################################################################################          
    def on_drop_features_button_clicked(self, b):
        if not self.df.empty:
            columns_to_drop = self.theSelectedInputFeaturesChoice.value 
            protected, non_protected, all_features = self.get_all_engineered_features()
            p = set(protected)# for fast reference operation

            for selected in columns_to_drop:
                if selected in p:# if protected
                    self.protected_feature_set_df.drop(feature, axis=1, inplace=True) 
                else: 
                    self.non_protected_feature_set_df.drop(feature, axis=1, inplace=True)
            with self.dropColOut:
                clear_output(wait=False)
                self.helper_methods.display_htmp ("Dropped "+ str (columns_to_drop),self.text_color, "h5")
                
    
    #################################################################################################
    #  
    # Return the data with or without the protected attributes for the purpose of training
    ################################################################################################# 
    def withOrWithoutProtected(self, selectProtected_value):
        try:

            print("Input Feature set to use:")
            if (selectProtected_value == 'Yes'):
                self.helper_methods.display_html (str(self.non_protected_feature_set_df.columns.values+
                                                     self.protected_feature_set_df.columns.values), self.text_color, "h5")
    
            
            if (selectProtected_value == 'No'):
                self.helper_methods.display_html (str(self.non_protected_feature_set_df.columns.values), self.text_color, "h5")

        except Exception as e:
            display(e)


    def get_feature_set(self):
        if (self.selectProtected.value == 'Yes'):
            return self.feature_set_final

        if (self.selectProtected.value == 'No'):
            return self.feature_set_final_without_protected

    def get_protected(self):
        '''Returns:
        - a list of protected Features
        - a dictionary of renamed groups in a protected Feature
        - a dictionary of descriptions for a protected Feature.'''
        return self.protected_feature_set_df.columns.values
        

    def get_X_y(self):
        return self.non_protected_feature_set_df, self.protected_feature_set_df, self.y_label_df
        
    
    
    def get_pre_process_info(self):
        return self.encoding_dict, self.renamed, self.referenceNames
    
    def high_positive(self):
            return self.HIGH_RANGE_POSITIVE
            
    def save_file_to(self, chooser):
        fc._select.description = "Save File as.."
        #Combine protected/non protected and y and save to csv
        df = pd.concat([self.non_protected_feature_set_df,self.protected_feature_set_df, self.y_label_df], axis=1, keys=['a', 'b', 'c'])
        df.to_csv(self.fc.selected_path + "/" + self.fc.selected_filename, index=False)#columns=['name']
        del df
    
    # Register callback function
        self.fc.register_callback(save_file_to)

    def render(self):
        
        box_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='90%')
        
    
        
        space = widgets.Label('  ', layout=widgets.Layout(width='100%'))

        uploadHTML = widgets.HTML(
            "<h3><left>1. Upload the data file to be used to train/test/validate your ML model :</left></h3>")
        
        removeRowsColumnsHTML = widgets.HTML(
            "<h3><left>2. Review and handle missing data:</left></h3> ")
        
        selectYHTML = widgets.HTML(
            """<h3><left>3. Select the Label(Y):</left></h3> 
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            I.e. The score that you want your final model to predict, bias can 
            cause an algorithm to miss the relevant causal relations between the input 
            features and this score(target output).""")
        
        
        selectImpactHTML = widgets.HTML(
            """<h3><left>4. Select an Impact the prediction will have on an individual:</left></h3>""")
        
        
        rankingEffectHTML = widgets.HTML(
            "<h4><center>Impact of output value</center></h4>")
        
        
        selectXHTML = widgets.HTML(
            """
            <h3><left>5. Select possible Input features and Protected Features</left></h3>
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            <b>Fairness and Disparate Treatment</b> — A decision making process suffers from disparate 
            treatment if the decision is even partly based on a protected Feature. It generally involves 
            an intent to discriminate, however in the case of ML this can occur when the model 
            detects a corrilation between the feature in question(such as gender, race, religion or age) 
            and the expected labeled output used during the training process. This type of relative 
            discrimination can be detected if a predicted outcome changes on changing the value of the
            protected Feature. 
            
            <br><b> Note:</b> Later you can choose to exclude these protected attributes during model 
            training, however it will be important to retain this information as it pertains to validation
            /test data in order to measure potential bias in the final model.</b>  
            """
        )
        
                   
        
        proxyFeaturesHTML = widgets.HTML(  """ 
            <h3><left>X. Select any features that might be proxy attributes for a protected group</left></h3>
        
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            <b>Fairness and Disparate Impact</b> — A decision making process suffers from disparate impact if 
            the outcome of a decision disproportionately hurts (or benefits) those within a group. No intent 
            is required. This type of relative discrimination can be detected if there is a difference in the
            fraction of positive (negative) outcomes for the different sensitive groups. Discrimination can
            happen for example through other correlated features such as zip codes, or any input feature that is
            itself generated by an algorithm (for example credit scores).
            <br><b>Note:</b> Disparate impact is often referred to as unintentional discrimination,
            whereas disparate treatment is intentional. 
            """)

        
        MLOutputFeaturesHTML =  widgets.HTML("""
            <h3><left>X. Select any features that might themselves have been generated from a 
            ML malgorithm</left></h3>
            <font style="font-family:sans-serif; font-size:15px;color:grey;">    
        """)
        
        
        ProtectedAttributesHTML = widgets.HTML(
            """<h3><left>6. Visualise protected attributes:</left></h3>
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
             - View the representation of protected groups in the dataset.
            <br> - View the distribution of protected groups in the dataset.
            <br> - View the proportion of positive outcomes in the data set per group.
            <br>This analysis can be used to determine if there is an imbalanced representation of protected 
            groups in the dataset.We have not yet defined a bias metric therefor we refer here only to
            balance of data as opposed to any bias this may cause.
            """)

        mergeFeaturesHTML = widgets.HTML(
            """<h3><left>7. Merge Categorical feature levels(unique values)). </left></h3> 
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            This functionality can be used to reduce the number of levels(unique values) per categorical 
            feature by merging levels. For example you could merge "race" levels to form {White, Non-White}
            or gender to {Male, Non-Male}. The main motivation for merging levels(unique values) is to make 
            the model more robust and to prevent overfitting, which may occur if one level is
            disproportionately represented in the training data, the model will not understand an 
            underrepresente level very well (nor will it generalize). One option is to make an explicit 
            "other" category for values. There may however be a cost to performance as every time you merge
            levels, you sacrifice information and make your data more regularized. 
            <p style="color:green">Note: This functionality can also be use to rename a categorical
            feature level.</p>""")
        
        storeReferenceForFeaturesHTML = widgets.HTML(
            """<h3><left>8. Store description of feature value(s)</left></h3> 
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            The reason we might want to do this is so that we can review the data and model for bias 
            without applying any personal bias during the process. 
            We can also do this to help with reverse one-hot-encode/label encode later. 
            This dictionary will be returned with the newly generated data file """)
        
        selectLEHTML = widgets.HTML(
            """<h3><left>9. Label-Encode <b>Ordinal</b> categorical features. </left></h3>
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            Categorical features are those that contain a finite number of label values rather than 
            numeric values, ordinal categories are those that have a natural ordered relationship between 
            each other which the model may be able to understand and harness. Label-Encoding can also 
            be applied to nominal categorical features if there are only two categories, 
            as this is equivalent to One-Hot-Encoding(with "first column drop" applied)""")
        
        selectHEHTML = widgets.HTML(
            """<h3><left>10. One-Hot-Encode <b>Nominal</b> categorical features </left></h3>
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            <b>Nominal</b> Categorical features are those that that have NO natural ordered relationship e.g 
            Gender: {Female, Male}, or Race: {Asian, Black, Hispanic, White}. 
            Keep in mind Multicollinearity and the "One-Hot-Encoding trap"(Dummy Variable Trap.) in the case of
            gender for example if there are only two genders defined, it would be enough to label encode to 1/0 
            (or 0/1) as this is the equivalent to one-hot encoding except using one input feature instead of 
            two, thus avoiding the "dummy variable trap, when using one-hot encoding, it is common to leave 
            out one of the columns because it can be inferred as being that which is absent 
            (i.e. if all of the other columns in the one-hot encoded dummy variables are 0, 
            then it must be the other category". This can be achieved in step 13 if required""")
        
    
        changeValueFeaturesHTML = widgets.HTML(
            """<h3><left>11. Change the data type of a feature </left></h3> 
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            e.g. Change an int to a string or a string to an int""")
        
        
        selectScaleHTML = widgets.HTML(
            """<h3><left>12. Scale/Normalise input features </left></h3>
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            Many machine learning algorithms work better when features are on a relatively
            similar scale and close to normally distributed. 
            MinMaxScaler, RobustScaler, StandardScaler, and Normalizer are scikit-learn methods to 
            preprocess data for machine learning.""")

        
        correlationsHTML = widgets.HTML(
            """<h3><left>13. Feature Correlation Visualisation</left></h3>
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            Discrimination/Bais canoccur through features that behave as proxies for a protected feature,
            for example through the correlation of features such as zip codes, or any input feature that is
            itself generated by an algorithm (for example credit scores).
            
            
            """)

        dropFeaturesHTML = widgets.HTML(
            "<h3><left>14. Drop additional Input Features</left></h3>")

        "The integer values have a natural ordered relationship between each other and machine learning algorithms may be able to understand and harness this relationship."
         

        selectUseProtected = widgets.HTML(
            "<h4><left>15. Include protected features in model training?</left></h4>")
        
        
        todo = """ <br>Fairness as a philosophy has no objective definition and therefor there is no consensus on a 
            mathematical formulation for fairness. 
            <br>Something we have to consider when training a Machine learning model which will make a
            decision that has a positive or negative conseqnence on a person or group is if the historic
            decisions (labels) would have differed, or if representation of a particular group in the data 
            would be different had discrimination not occured in the pas. Is a loss in <b>percieved accuracy</b> 
            an acceptable trade off in an attempt to correct historic discrimination </font>"""
        
        
        self.selectYFeature.layout.grid_area = 'topl'
        self.selectXFeatures.layout.grid_area = 'topl' 
        self.selectProtectedAttributes.layout.grid_area = 'topr'
        
        
        self.theSelectedInputFeaturesDropDown.layout.grid_area = 'topl'
        self.choosenFeatureValues.layout.grid_area  = 'topr'
        self.newNameForValue.layout.grid_area = 'middle'
        self.merge_values_button.layout.grid_area = 'bottom'

        
        self.theSelectedInputFeaturesDropDown.layout.grid_area = 'topl'
        self.descriptionText.layout.grid_area = 'middle'
        self.save_description_button.layout.grid_area = 'bottom'
        
        
        self.missingDataColumnsThresholdSlider.layout.grid_area = 'topl'
        self.missingDataRowsThresholdSlider.layout.grid_area = 'topr'
        self.remove_rows_cols_button.layout.grid_area = 'middle'
        self.selectColumnWithMissingData.layout.grid_area = 'topl'
        self.apply_imputation_button.layout.grid_area = 'bottoml'
        self.selectImputeMethod.layout.grid_area = 'topr'
        
        
        self.selectColumnType.layout.grid_area = 'topr'
        self.change_column_type_button.layout.grid_area = 'middle'
        
        self.selectXFeatures.layout.grid_area = 'topl'
        self.theSelectedInputFeaturesChoice.layout.grid_area = 'topl'
        self.theSelectedInputFeaturesDropDown.layout.grid_area = 'topl'
        self.view_attributes_button.layout.grid_area = 'topl'  
        self.label_encode_button.layout.grid_area = "topr"
        self.one_hot_encode_button.layout.grid_area =  "topr"
        self.selectColumnType.layout.grid_area  = "topr"
        self.change_column_type_button.layout.grid_area = "middlel"
        self.selectScaleType.layout.grid_area = "topr"
        self.apply_scale_button.layout.grid_area = "middlel"
                               
        self.view_correlation_button.layout.grid_area = "middlel"
        self.progressBar.layout.grid_area = "middle"
       
#         self.results_accordion.layout.grid_area = "bottom"
        self.drop_features_button.layout.grid_area = "bottom"
        
        
        display(widgets.VBox([
            uploadHTML,
            self.fileUploader,
            self.uploadFileOutput,
            
            ###########Missing Data#################
            space,
            removeRowsColumnsHTML,
            widgets.GridBox(children=[self.missingDataColumnsThresholdSlider,
                                     self.missingDataRowsThresholdSlider,
                                     self.remove_rows_cols_button,
                                     ],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto auto',
                        grid_template_columns='50% 50%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topr"
                        "middle ."
                        ''')
                   ),
            self.dropOutput,
            space,
            widgets.GridBox(children=[
                                     self.selectColumnWithMissingData,
                                     self.selectImputeMethod,
                                     self.apply_imputation_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto auto',
                        grid_template_columns='50% 50%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topr"
                        "bottoml ."
                        ''')
                   ),
            self.imputeOutput,
            
            
            ###########Select Label (Y)#################
            space,
            selectYHTML,
            widgets.GridBox(children=[self.selectYFeature],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto',
                        grid_template_columns='50% 50%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl ."
                        ''')
                   ),        
            self.selectLabelYOutput,
            
            ###########Select Impact of Score#################
            space,
            selectImpactHTML,
            self.selectImpact,
            self.selecImpactOutput,
            
            
            ############Select Protected Attribute################
            space,
            selectXHTML,
            widgets.GridBox(children=[self.selectXFeatures, self.selectProtectedAttributes],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto',
                        grid_template_columns='50% 50%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topr"
                        ''')
                   ),   
            self.selectLabelXOutput,
            
            
            ############View protected attribute stats################
            space,
            ProtectedAttributesHTML,
            widgets.GridBox(children=[self.view_attributes_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto auto',
                        grid_template_columns='50% 50%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topl"
                        ''')
                   ),   
            
            self.viewProtectedOut,
            
            
            
            ##########Merge Feature values##################
            space,
            mergeFeaturesHTML,
            widgets.GridBox(children=[self.theSelectedInputFeaturesDropDown, self.choosenFeatureValues, self.newNameForValue, self.merge_values_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto auto auto',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topr"
                        "middle ."
                        "bottom ."
                        ''')
                   ),        
            self.mergeOut,
            
            
            ############Store Description################
            space,
            storeReferenceForFeaturesHTML,
            widgets.GridBox(children=[self.theSelectedInputFeaturesDropDown, self.choosenFeatureValues, self.descriptionText, self.save_description_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto auto auto',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topr"
                        "middle ."
                        "bottom ."
                        ''')
                   ),   
            self.saveDescriptionOut,
            
            ###########Label Encode#################  
            space,
            selectLEHTML,
            widgets.GridBox(children=[self.theSelectedInputFeaturesChoice, self.label_encode_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topr"
                        ''')
                   ),
            self.theSelectedInputFeaturesChoiceOut,
            self.LEOut,
            
            
            ###########One-Hot-Encode#################
            space,
            selectHEHTML,
            widgets.GridBox(children=[self.theSelectedInputFeaturesDropDown, self.one_hot_encode_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topr"
                        ''')
                   ), 
            self.theSelectedInputFeaturesDropDownOut,
            self.HotEncodeOut,
            
    
            
            
            ###########Change Type#################
            space,
            changeValueFeaturesHTML,
            widgets.GridBox(children=[self.theSelectedInputFeaturesDropDown, self.selectColumnType, self.change_column_type_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto auto ',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topr"
                        "middlel ."
                        ''')
                   ),    
            self.theSelectedInputFeaturesDropDownOut,
            self.changeDataTypeOut,
            
            
            ###########Scale/Normalise Features#################
            space,
            selectScaleHTML,
            widgets.GridBox(children=[self.theSelectedInputFeaturesChoice, self.selectScaleType, self.apply_scale_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topr"
                        "middlel ."
                        ''')
                   ),   
            self.theSelectedInputFeaturesChoiceOut,
            self.scaleNormaliseOut,
            
            ###########View Correlation#################
            space,
            correlationsHTML,
            # self.show_correlation_btn,
            widgets.GridBox(children=[self.theSelectedInputFeaturesChoice, self.view_correlation_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto, auto',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl ."
                        "middlel ."
                        ''')
                   ),    
            self.correlationsOut,
            self.correlationsGraphOut,
            
            
            ##########Drop Features##################
            space,
            dropFeaturesHTML,
            widgets.GridBox(children=[self.theSelectedInputFeaturesChoice,self.drop_features_button ],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto auto',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl ."
                        "bottom ."
                        ''')
                   ), 
            self.dropColOut,
            
            
            ##########Are you aware of possible proxy features##################
            space,
            proxyFeaturesHTML,
            widgets.GridBox(children=[self.theSelectedInputFeaturesChoice],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl ."
                        ''')
                   ), 
           
            
            ##########Are any of the features an output from a ML algorithm##################
            space,
            MLOutputFeaturesHTML,
            widgets.GridBox(children=[self.theSelectedInputFeaturesChoice ],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl ."
                        ''')
                   ), 
            self.dropColOut,
            
            ############Use protected in Training################
            space,
            selectUseProtected,
            widgets.VBox([self.selectProtected, self.protectedOut])
        ]))
        
        display (self.fc)
        
        


In [6]:
dpUI = data_pre_process_UI()
dpUI.render()
dpUI.df

FileChooser(path='/Users/aideenf/Documents/GitHub/Thesis_Aideen/Main', filename='data.csv', show_hidden='True'…

""


In [ ]:
display (dpUI.df.head())

In [ ]:
display (dpUI.protected_feature_set_df.head(5))
display (dpUI.non_protected_feature_set_df.head(5))
display (dpUI.renamed)
display (dpUI.referenceNames)
display (dpUI.df_url)
display (dpUI.encoding_dict)


         
# a, b, c = dpUI.get_X_y()
# return self.non_protected_feature_set_df, self.protected_feature_set_df, self.y_label_df
# display (a,b,c)       
    
    
# d, e, f = dpUI.get_pre_process_info()
# return self.encoding_dict, self.renamed, self.referenceNames
         
# display (d,e,f)


In [ ]:
!pip install --upgrade aequitas
!pip install --upgrade pandas


In [ ]:
!pip install --upgrade pandas-profiling


In [ ]:
display(dpUI.non_protected_feature_set_df)

In [ ]:
# display (dpUI.non_protected_feature_set_df.head(5))
profile = pandas_profiling.ProfileReport(dpUI.non_protected_feature_set_df)
# 

In [ ]:
profile.to_widgets()

In [ ]:
feature_set_with_y = dpUI.get_dataframe(include_label_y = True, include_protected = True)[0]
column_list = list(dpUI.theSelectedInputFeaturesChoice.value)
column_list_with_Y = list(dpUI.theSelectedInputFeaturesChoice.value)+[dpUI.selectYFeature.value]
protected_column_list = dpUI.protected_feature_set_df.columns                  
                    
dpUI.helper_methods.view_correlations(feature_set_with_y, column_list, column_list_with_Y, protected_column_list, dpUI.selectYFeature.value )  




# Run Pandas Profiling on the final input feature set
- Later might incorporate the relevant/useful features of pandas profiling into the UI

In [ ]:
#Run pandas profile report on the final input feature set
display (dpUI.get_feature_set().head(5))

#NOTE SOMETIMES HAVE TO RUN THIS TWICE
#NEED TO DEBUG
pandas_profiling.ProfileReport(dpUI.get_feature_set())



# Obtain the input features and label data

We should convert a pandas DataFrame into a NumPy array in order to train a machine learning model on it. 
Being careful to remove the target variable from the data before training a model(this is taken care of by
the user interface) 

Note:Including the output variable, or some derived property of the output variable, into the feature representation
is a very common mistake in building supervised machine learning models.

In [ ]:
ProtectedAttributes, renamedGroups, referenceNamesforGroups = dpUI.get_protected()
highRankingIsPositive = dpUI.high_positive ()

X, y = dpUI.get_X_y()

display(X.head(1))
display(y.head(1))

# Create the train/test split. 

*arrays : sequence of indexables with same length / shape[0]
Allowed inputs are lists, numpy arrays, scipy-sparse matrices or pandas dataframes.

test_size : float, int or None, optional (default=None)
If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split. If int, represents the absolute number of test samples. If None, the value is set to the complement of the train size. If train_size is also None, it will be set to 0.25.

train_size : float, int, or None, (default=None)
If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the train split. If int, represents the absolute number of train samples. If None, the value is automatically set to the complement of the test size.

random_state : int, RandomState instance or None, optional (default=None)
If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.

shuffle : boolean, optional (default=True)
Whether or not to shuffle the data before splitting. If shuffle=False then stratify must be None.

stratify : array-like or None (default=None)
If not None, data is split in a stratified fashion, using this as the class labels.

In [ ]:
#Create the train/test split. 
#X_train and X_test are pandas.core.frame.DataFrame 
#y_train and y_test are pandas.core.series.Series 

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0)



In [ ]:
display("First X_train record:")
display("****************************************")
display(X_train.head(1))

display("First X_test record:")
display("****************************************")
display(X_test.head(1))

# Linear Regression - Regression Model
Remember, a linear regression model in two dimensions is a straight line; in three dimensions it is a plane, and in more than three dimensions, a hyperplane. Here we have multiple dimensions or input variables so our model will be a “multiple linear regression” or "multivariate linear regression". It would be possible to predict the exact FYA by making this a regression task.

In [ ]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# SHAP Library

In [ ]:
my_model = SomeModelClass()
my_model.fit(X_train, y_train)
explainer = shap.KernelExplainer(my_model.predict, X_train.iloc[0:100]) # The second argument is the "background" dataset; a size of 100 rows is gently encouraged by the code
x = X_test.iloc[samples_to_explain]
kernel_shap = explainer.shap_values(x)

In [ ]:
#We will be using Root mean squared error(RMSE) and Coefficient of Determination(R² score) to evaluate our model.
#RMSE is the square root of the average of the sum of the squares of residuals. 
#The RMSE is the square root of the variance of the residuals. 
#It indicates the absolute fit of the model to the data–how close the observed data points are to the model’s
#predicted values. Whereas R-squared is a relative measure of fit, RMSE is an absolute measure of fit. 
#As the square root of a variance, RMSE can be interpreted as the standard deviation of the unexplained variance, 
#and has the useful property of being in the same units as the response variable. 
#Lower values of RMSE indicate better fit. 
#RMSE is a good measure of how accurately the model predicts the response, and it is the most important criterion
#for fit if the main purpose of the model is prediction.

#Coefficient of Determination(R² score) - The best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). 
#A constant model that always predicts the expected value of y, disregarding the input features, 
#would get a R^2 score of 0.0. #R-squared is between 0 and 1, Higher values are better because it means 
#that more variance is explained by the model.

# printing values
print("")
print('Slope:' ,linreg.coef_)
print("")
print('Intercept:', linreg.intercept_)
print("")
display("*****EVALUATING MODEL WITH TRAINING DATA:***")
y_predictedTrain = linreg.predict(X_train)
rmse = mean_squared_error(y_train, y_predictedTrain)
r2 = r2_score(y_train, y_predictedTrain)
display(' Root mean squared error: '+ str(rmse))
display(' R2 score: '+ str(r2))

display("*****EVALUATING MODEL WITH TEST DATA:******")
y_predictedTest = linreg.predict(X_test)
rmse = mean_squared_error(y_test, y_predictedTest)
r2 = r2_score(y_test, y_predictedTest)
display(' Root mean squared error: ' + str(rmse))
display(' R2 score: '+ str(r2))


In [ ]:
n_train = len(X_train)
import matplotlib.pyplot as plt
#y_pred_train = linreg.predict(X_train)
#y_pred_test = linreg.predict(X_test)
plt.figure(figsize=(20, 10))
plt.plot(range(n_train), y_train, label="train")
plt.plot(range(n_train, len(y_test) + n_train), y_test, '-', label="test")
plt.plot(range(n_train), y_predictedTrain, '--', label="prediction train")

plt.plot(range(n_train, len(y_test) + n_train), y_predictedTest, '--',
             label="prediction test")
plt.legend(loc=(1.01, 0))
plt.xlabel("Score")
plt.ylabel("prediction")

# Now Rank the Test data results according to the score
here we have the Test input data, the actual value and the predicted value. for the purpose of the ranking exercise we will use the test data and the value predicted by the model 

In [ ]:
y_predictedTestDF = pd.DataFrame(y_predictedTest)
y_predictedTestDF  = y_predictedTestDF.rename(columns={0: "Predicted"})
#display((y_predictedTestDF)[0:1])


#print ("X-Test Shape" ,X_test.shape)
#print ("y-Test Predicted Shape" ,y_predictedTestDF.shape)
df_for_ranking = X_test.reset_index()
df_for_ranking = df_for_ranking.merge(y_predictedTestDF, left_index=True, right_index=True)
ranked_df = df_for_ranking.sort_values(by ='Predicted',  ascending=False )
ranked_df = ranked_df.drop("index", axis=1)
ranked_df = ranked_df.reset_index()
ranked_df = ranked_df.drop("index", axis=1)

display((ranked_df)[0:5])
print (ranked_df.shape)

# Analise with IBM 360 + Aequitas (Todo)

https://towardsdatascience.com/a-tutorial-on-fairness-in-machine-learning-3ff8ba1040cb
https://arxiv.org/pdf/1908.00176.pdf
https://5harad.com/papers/fair-ml.pdf


Notes "Fairness is a complex concept, with a number of different possible definitions. We consider group fairness with respect to membership in a protected group, based, for example, on minority race or underrepresented gender, where group membership is readily ascertained by looking at an attribute value. For a given rank cut-off point k, we wish to ensure that the number of protected group members ranked among the top-$k$ is proportional to their representation in the entire population, or to their desired proportion in the output (as is the case in affirmative action interventions)."


To-Do
#Create new protected attributes based on a combination of the other protected 
            #attributes so for example if you belong to the protected Group A of Attribute A
            #and protected grouo B of attribute B then you are in a double protected group
            #In order to achieve this we must provide the possibility of identifying 
            #the protected group for each protected attribute.
            
            #df['multi_protected'] = df.apply(lambda row: row.a and row.b, axis=1)   


https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6
    

In [ ]:
class review_fairness_UI():

    def __init__(self, **kwargs):

        self.ranked_df = kwargs['ranked_df']
        self.protected_attributes = kwargs['protected_attributes']
        self.group_names = kwargs['group_names']
        self.renamed_groups = kwargs['renamed_groups']

        self.high_ranking_positive = kwargs['high_ranking_positive']
        self.HIGH_RANGE_POSITIVE = 1
        if self.high_ranking_positive == False:
            self.HIGH_RANGE_POSITIVE = 0

        self.style = {'description_width': 'initial'}
        self.layout = {'width': '700px'}
        self.layout_short = {'width': '300px'}
        
        self.dictOfSlidersPercentOfPop = {}
        self.dictOfSlidersPercentOfDomain = {}
        self.dictOfSlidersAffermativeActionGoal = {}

        # Either upload the file or create the class with pandas Dataframe as input
        self.fileUploader = FileUpload(accept='.csv',
                                       multiple=False,
                                       disabled=False,
                                       style_button='color: darkblue; background-color: lightsalmon; width: 180px;',
                                       compress_level=9
                                       )


        self.selectYAttribute = widgets.Dropdown(
            options=self.ranked_df.columns.values,
            value="Predicted",
            description='Ranked Output:',
            disabled=False,
            style=self.style,
            layout=self.layout
        )

        self.selectImpact = widgets.RadioButtons(
            options={'A Positive impact on the the life of the individual i.e Assistive': 1,
                     'A negative impact on the the life of the individual i.e Punitive': 0},
            description='For decisions, a high rank will results in:',
            disabled=False,
            layout=self.layout,
            style=self.style,
            value=self.HIGH_RANGE_POSITIVE
        )

        self.percentOrTopK = widgets.RadioButtons(
            options={'Top-K results': 1,
                     'Top-X percent of results': 0},
            description='For analysis use:',
            disabled=False,
            layout=self.layout,
            style=self.style,
            value=1
        )

        def on_impact_selected(change):
            if (change['new'] == 1):
                self.HIGH_RANGE_POSITIVE = 1
            if (change['new'] == 0):
                self.HIGH_RANGE_POSITIVE = 0
        self.selectImpact.observe(on_impact_selected, names='value')

        # use to define the selected attributes, one or more
        self.selectProtectedAttributes = widgets.SelectMultiple(
            options=self.ranked_df.columns.values,
            value=self.protected_attributes,
            description='Protected Attribute(s)',
            disabled=False,
            style=self.style,
            layout=self.layout
        )

        # use to select the protected attribute to examine
        self.selectProtectedAttributeToExamine = widgets.Dropdown(
            description='Protected Attributes',
            disabled=False,
            style=self.style,
            layout=self.layout
        )

        self.choosen_attribute_values = widgets.SelectMultiple(
            description='Select attribute value',
            disabled=False,
            style=self.style,
            layout=self.layout_short
        )

        self.selectFairnessMeasure = widgets.Dropdown(
            options=['Counterfactual fairness', 'Equality of outcome', '3'],
            # rows=10,
            description='Select the definition of fairness that best applies',
            disabled=False,
            style=self.style,
            layout=self.layout
        )
        
        self.selectFairnessGoal = widgets.Dropdown(
            options={'Equal numbers from each group':1, 'Proportional to representation in population':2, 'Proportional to representation in domain':3, 'Affirmative action for representation in domain':4},
            description='Fairness goal',
            disabled=False,
            style=self.style,
            layout=self.layout
        )

        # Exposure Drop Measure - Relating to the concept of uneven distribution of attention
        self.selectExpDropMeasure = widgets.Dropdown(
            options={'Standard logarithmic discount (using sum)':1, 'Standard logarithmic discount (using average)':2, 'Geometric Distribution - Biega et al (TBD)':3, 'Viable-n test(TBD)':4},
            # rows=10,
            description='Exposure Drop-off (Position Bias)',
            disabled=False,
            style=self.style,
            layout=self.layout
        )
        
        

        self.selectPercentageOfResults = widgets.Dropdown(
            options=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
            # rows=10,
            description='Top-X percent of results: ',
            disabled=False,
            style=self.style,
            layout=self.layout
        )

        self.selectTopKResults = widgets.IntText(
            value=20,
            description='Top-K results:            ',
            disabled=False,
            style=self.style,
            layout=self.layout,
            max=1000,
            min=0,
        )
        
        self.setSignificance = widgets.FloatSlider(
            value=0.1,
            min=0,
            max=1.0,
            step=0.1,
            description='Significance paramater (a):',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
            tooltip = 'A large value means a larger probability of declaring a fair ranking unfair',
            style=self.style,
            layout=self.layout
    )



        self.outUploadData = widgets.interactive_output(self.upload_data, {
            'fileUploader_value': self.fileUploader})

        self.outSelectProtected = widgets.interactive_output(self.select_protected, {
            'selectProtectedAttributes_values': self.selectProtectedAttributes})

        self.outSelectRankedListCol = widgets.interactive_output(self.select_ranked_column, {
            'selectYAttribute_values': self.selectYAttribute})

        self.viewProtectedOut = widgets.interactive_output(self.view_protected, {
            'selectProtectedAttributes_values': self.selectProtectedAttributes,
            'selectYAttribute_values': self.selectYAttribute})

        self.viewAnalysisOut = widgets.interactive_output(self.run_analysis, {
            'selectProtectedAttributeToExamine_value': self.selectProtectedAttributeToExamine,
            'selectPercentageOfResults_value': self.selectPercentageOfResults,
            'selectExpDropMeasure_value': self.selectExpDropMeasure,
            'selectTopKResults_value': self.selectTopKResults,
            'percentOrTopK_value': self.percentOrTopK,
            'selectFairnessGoal_value': self.selectFairnessGoal,
            'setSignificance_value': self.setSignificance})

        self.viewAnalysisGroupPercentInPop = widgets.interactive_output(self.create_population_slider, {
            'selectProtectedAttributeToExamine_value': self.selectProtectedAttributeToExamine})
        
        self.viewListInfoOut = widgets.Output(layout={})
        self.viewProtectedPandasPlotA = widgets.Output(layout={})
        self.viewFairOutputA = widgets.Output(layout={})
        

        # to populate the UI at startup - add this to a method later
        self.select_protected(self.selectProtectedAttributes.value)
        self.select_ranked_column(self.selectYAttribute.value)
        self.view_protected(self.selectProtectedAttributes.value, self.selectYAttribute.value)
        
        self.run_analysis(
                     self.selectProtectedAttributeToExamine.value,
                     self.selectPercentageOfResults.value,
                     self.selectExpDropMeasure.value,
                     self.selectTopKResults.value,
                     self.percentOrTopK.value,
                     self.selectFairnessGoal.value,
                     self.setSignificance.value
                     )

        #### # # # # # # # # # # # # # # 

    def upload_data(self, fileUploader_value):
        if not(self.ranked_df.empty):
            display("Ranked list already imported")
            display(self.ranked_df.head(5))
        try:
            for key in fileUploader_value:
                display(HTML("uploaded: ",
                      fileUploader_value[key]['metadata']['name']))
                try:
                    csvInBytes = fileUploader_value[key]['content']
                    s = str(csvInBytes, 'utf-8')
                    data = StringIO(s)
                    self.ranked_df = pd.read_csv(data)
                    csvInBytes = 0
                    data = 0
                    s = 0
                    print("")
                    print("Snapshot of unmodified data set")
                    display(self.ranked_df.head(5))
                except:
                    print("Error reading file - upload new file")
        except:
            print("Error uploading file - upload again")

    
    def select_protected(self, selectProtectedAttributes_values):
        try:
            with self.outSelectProtected:
                clear_output(wait=True)
                display(HTML("<b><font color='green'> Protected Attributes </b> </font>"),
                        selectProtectedAttributes_values) 
                self.selectProtectedAttributeToExamine.options = selectProtectedAttributes_values
                self.selectProtectedAttributeToExamine.value = selectProtectedAttributes_values[0]    
        except:
            display("Protected attributes not yet set")

    
    
    def select_ranked_column(self, selectYAttribute_values):
        try:
            with self.outSelectRankedListCol:
                clear_output(wait=True)
                display(HTML(
                    "<b><font color='green'> Ranked output column </b> </font>"), selectYAttribute_values)

        except:
            display("Ranked column not yet set")

   
    def view_protected(self, selectProtectedAttributes_values, selectYAttribute_values):
        %matplotlib inline
        protected = []
        protectedCount = []
        protectedPercentage = []

        try:
            with self.viewProtectedOut:
                clear_output(wait=True)
                colType = self.ranked_df[selectYAttribute_values].dtype
                if (colType != str and colType != object):
                    yMin = self.ranked_df[selectYAttribute_values].min()
                    yMean = self.ranked_df[selectYAttribute_values].mean()
                    yMax = self.ranked_df[selectYAttribute_values].max()

                    
                    bins = [yMin, yMean, yMax]
                    
                    yMin = round(yMin, 3)
                    yMean = round(yMean, 3)
                    yMax = round(yMax, 3)              
                    newColName = selectYAttribute_values + "-range"
                    names = ['', '']
                    
                    if self.selectImpact.value == 1:
                        names = ['Neg_Outcome', 'Pos_Outcome']
                    # min to mean is neg, mean to max is pos
                    
                    if self.selectImpact.value == 0:
                        names = ['Pos_Outcome', 'Neg_Outcome']
                    # min to mean is pos, mean to max is neg

                    
                    self.ranked_df[newColName] = pd.cut(
                        self.ranked_df[selectYAttribute_values], bins, labels=names)
                    x = "<br>* Output '" + \
                        str(selectYAttribute_values) + "' grouped in range:"
                    text = text +  names[0]+ " -> Above average relevance, defined as Mean to Max value = " + str(yMean) + " to " + str(yMax)+"<br>"
                    text = text + names[1]+ " -> Below average relevance, defined as Min to Mean value = " + str(yMin) + " to " + str(yMean)
                    display(HTML(text))
                    display(HTML("<br><b>* Impact:</b>"))
                    if self.HIGH_RANGE_POSITIVE == 1:
                        impactTxt = "<b>positive</b>"
                    if self.HIGH_RANGE_POSITIVE == 0:
                        impactTxt = "<b>negative</b>"
                    x = "The Impact of a high ranking on an individual or group is " + impactTxt
                    display(HTML(str(x)))
                    display(HTML("<br><b>* Analysis of protected attributes (Ground truth):</b>"))


                    # see https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
                    for feature in selectProtectedAttributes_values:
                        # print(feature.upper(), ':')
                        protected.append(feature)
                        # protectedCount and protectedPercentage are array of type pd.Series
                        protectedCount.append(
                            self.ranked_df[feature].value_counts())
                        protectedPercentage.append(
                            self.ranked_df[feature].value_counts(normalize=True) * 100)

                    if (len(protected) > 0):

                        for (PDSeries, PDSeriesPcnt, PDName) in zip(protectedCount, protectedPercentage, protected):
                            
                            df = pd.DataFrame(
                                dict(Count=PDSeries, Percentage=PDSeriesPcnt)).reset_index()
                            df = df.rename(
                                columns={"index": PDName.capitalize()})
                            df = df.style.set_properties(**{'background-color': 'black',
                                                            'color': 'lawngreen',
                                                            'border-color': 'white'})

                            display(HTML("<b><font color='green'>" + PDName + ":</b></font>"))
                                                
                            display (self.group_names[PDName])
                            display(self.renamed_groups[PDName])
                            display(df)

                            #Create two pandas plots and display them with matplot
                            plt.tight_layout();
                            fig, (ax, ax2) = plt.subplots(ncols=2, sharey=False);
                            fig.set_figheight(5);
                            fig.set_figwidth(10);
                            fig.subplots_adjust(wspace=0.4, bottom=0.3);
                            plotA = PDSeries.plot.pie(title="Representation of groups in dataset",ax=ax,autopct='%.2f');
                            plotB = pd.crosstab(ranked_df[PDName], ranked_df[newColName]).plot.barh(stacked=False, title="Distribution of outcome in data set",ax=ax2);
                            plt.show();


            if (colType == str or colType == object):
                print("Output label is not a number and should be converted")
        except:
            print("**Protected groups visualization not yet generated**")

            
            
            
            
    def create_population_slider(self,
                                selectProtectedAttributeToExamine_value
                             ):
        try:
            allGroupsInProtected = self.ranked_df[selectProtectedAttributeToExamine_value].unique()
            result = self.group_names[selectProtectedAttributeToExamine_value]
            with self.viewAnalysisGroupPercentInPop:
                clear_output(wait=True)
                try:
                    for group in allGroupsInProtected:
                        desc = str(group) + " : " + str(result[str(group)]) + ": Percentage of population"
                        desc2 = str(group) + " : " + str(result[str(group)]) + ": Percentage of domain"
                        desc3 = str(group) + " : " + str(result[str(group)]) + ": Percentage goal (Affermitive action)"
                        
                        self.dictOfSlidersPercentOfPop[group] = widgets.IntSlider(
                                value=0,
                                description= desc ,
                                max=100,
                                min=0,
                                style=self.style,
                                layout=self.layout)
                        
                        self.dictOfSlidersPercentOfDomain[group] = widgets.IntSlider(
                                value=0,
                                description= desc2 ,
                                max=100,
                                min=0,
                                style=self.style,
                                layout=self.layout)
                        
                        self.dictOfSlidersAffermativeActionGoal[group] = widgets.IntSlider(
                                value=0,
                                description= desc3 ,
                                max=100,
                                min=0,
                                style=self.style,
                                layout=self.layout)
        
                except:

                    for group in allGroupsInProtected:
                        desc = str(group) + " : " + str(result[group]) + ": Percentage of population"
                        desc2 = str(group) + " : " + str(result[group]) + ": Percentage of domain"
                        desc3 = str(group) + " : " + str(result[group]) + ": Percentage goal (Affermitive action)"
                        
                        self.dictOfSlidersPercentOfPop[group] = widgets.IntSlider(
                                value=0,
                                description= desc ,
                                max=100,
                                min=0,
                                style=self.style,
                                layout=self.layout) 
                        self.dictOfSlidersPercentOfDomain[group] = widgets.IntSlider(
                                value=0,
                                description= desc2 ,
                                max=100,
                                min=0,
                                style=self.style,
                                layout=self.layout)
                        
                        self.dictOfSlidersAffermativeActionGoal[group] = widgets.IntSlider(
                                value=0,
                                description= desc3 ,
                                max=100,
                                min=0,
                                style=self.style,
                                layout=self.layout)
                        
                display (HTML("<font color='blue'> leave value = 0 if no action is to be taken for a particular group</font>"))
                            
                for group in allGroupsInProtected:
                    display (self.dictOfSlidersPercentOfPop[group])
                    display (self.dictOfSlidersPercentOfDomain[group])
                    display (self.dictOfSlidersAffermativeActionGoal[group])
        except:
            print("output not generated yet")  
        
        
    def run_analysis(self,
                     selectProtectedAttributeToExamine_value,
                     selectPercentageOfResults_value,
                     selectExpDropMeasure_value,
                     selectTopKResults_value,
                     percentOrTopK_value,
                     selectFairnessGoal_value,
                     setSignificance_value
                     
                     ):
        expList = []

        local_ranked_df = self.ranked_df.copy()
        
        if self.percentOrTopK.value == 0:
             # Now reduce the ranked list based on whethere we are using percentage or top k
                resultsLen=round(len(local_ranked_df) *selectPercentageOfResults_value/100)
        elif self.percentOrTopK.value == 1:
            # Now reduce the ranked list based on whethere we are using percentage or top k
                resultsLen= self.selectTopKResults.value
        
        
        try:
            with self.viewListInfoOut:
                clear_output(wait=True)
                textOne = 'Total number of records in ranked list (n) = ' + str(len(local_ranked_df))
                textTwo ='Note: when n is sufficiently small we expect the user to read all results or for all outcomes to be equal'
                display(HTML("<font color='blue'>" + textOne + " </font>"))
                display(HTML("<font color='blue'>" + textTwo + " </font>"))
                
        except:
            display ('Data not yet loaded')
        
        
        if (selectExpDropMeasure_value == 1):
            #Calculate the Logrithmic discount exposure drop off for the entire dataset 
            #and use the average in the calculations        
            for x in range(len(local_ranked_df)):
            # For each record add the exposure to a pandas Series
                exposure=1/math.log(x+2, 2)
                expList.append(exposure)
                
        elif (selectExpDropMeasure_value == 2):
            #Calculate the Logrithmic discount exposure drop off for the entire dataset 
            #and use the sum in the calculations 
            for x in range(len(local_ranked_df)):
            # For each record add the exposure to a pandas Series
                exposure=1/math.log(x+2, 2)
                expList.append(exposure)
                
        elif (selectExpDropMeasure_value == 3):
            #Calculate the geometric distribution exposure drop off for the entire dataset
            for x in range(len(local_ranked_df)):
            # For each record add the exposure to a pandas Series
                exposure=1/math.log(x+2, 2)
                expList.append(exposure)
        
        elif (selectExpDropMeasure_value == 4):
            #Calculate the Logrithmic discount exposure drop off for the entire dataset
            for x in range(len(local_ranked_df)):
            # For each record add the exposure to a pandas Series
                exposure=1/math.log(x+2, 2)
                expList.append(exposure)
                
        else:
            for x in range(len(local_ranked_df)):
            # For each record add the exposure to a pandas Series
                exposure=1/math.log(x+2, 2)
                expList.append(exposure)
                       
        # Now we have a full list with the "logarithmic discount" exposure
        local_ranked_df.insert(0, "exposure", expList, True)
        ranked_df_top=local_ranked_df.head(resultsLen).copy()
        #now we have a local copy of the full ranked list and the reduced ranked list with exposure 
        
        # just clear the viewAnalysisOutput as we are about to use it again
        
        try:
            with self.viewAnalysisOut:
                clear_output(wait=True)
                adjustment = 0
                lowestUtil = self.ranked_df[self.selectYAttribute.value].min()
        
                if (lowestUtil < 0):
                    display(HTML("<font color='blue'><b>Note: </b>Relevance score adjusted by min value " + str(round(lowestUtil,3)) + " to ensure no negative numbers in measure</font>") )
  
                adjustment = lowestUtil
                local_ranked_df['Predicted'] = local_ranked_df['Predicted'].apply(lambda x: x-adjustment)
        
                if (selectExpDropMeasure_value == 1):
                    #Create a small data frame containing the average exposure and relevance per group
                    fullExp_util=local_ranked_df.groupby(selectProtectedAttributeToExamine_value).sum()[
                        ['exposure', 'Predicted']]
                    reducedExp_util=ranked_df_top.groupby(selectProtectedAttributeToExamine_value).sum()[['exposure', 'Predicted']]
        
                
                elif (selectExpDropMeasure_value == 2):
                    #Create a small data frame containing the sum of exposure and relevance per group
                    fullExp_util=local_ranked_df.groupby(selectProtectedAttributeToExamine_value).mean()[
                        ['exposure', 'Predicted']]
                    reducedExp_util=ranked_df_top.groupby(selectProtectedAttributeToExamine_value).mean()[['exposure', 'Predicted']]
        
                     
                #else  #3, 4, 5 to be implemented 
                     
                fullExp_util['exposure_over_relevance']=fullExp_util.apply(
                    lambda row: row.exposure/row.Predicted, axis=1)
        
                fullExp_util.rename(columns={'Predicted':'relevance'}, inplace=True)
    
                display (HTML("Full dataset:"))
                display (self.group_names[selectProtectedAttributeToExamine_value])
                display(fullExp_util)

                print(" ")
                display(HTML("List reduced to top: " + str (resultsLen)))
                display (self.group_names[selectProtectedAttributeToExamine_value])
                # Create a ranked list of only the "top" records.
        
                reducedExp_util['exposure_over_relevance']=reducedExp_util.apply(
                    lambda row: row.exposure/row.Predicted, axis=1)
        
                reducedExp_util.rename(columns={'Predicted':'relevance'}, inplace=True)
                display(reducedExp_util)
                #reducedExp_util.plot.bar(stacked=False, figsize=(5, 5))

                self.view_protected_attribute (ranked_df_top, selectProtectedAttributeToExamine_value )

        except :
            print ("data not yet generated")
            
        try:
            #Populate the input to the FA*IR Analysis
            with self.viewFairOutputA:
                clear_output(wait=True)
                display(HTML("Larger values for significance(a) indicate a stricter necessity to adhere to the required porportion of protected elements at each position"))
                
                display(self.group_names[selectProtectedAttributeToExamine_value])
                allGroupsInProtected = self.ranked_df[selectProtectedAttributeToExamine_value].unique()
                for group in allGroupsInProtected:
                    display(HTML("Required percentage for group "+ str(group) +" is " + str (self.dictOfSlidersAffermativeActionGoal[group].value)))
                
                display(HTML("K =: " + str (resultsLen)))
                for group in allGroupsInProtected:
                    display(HTML("Targeted minimum proportion(p) for group "+ str(group) +" is " + str (self.dictOfSlidersAffermativeActionGoal[group].value/100)))
                
                display(HTML("a =: " + str (self.setSignificance.value)))
        except :
            print ("data not yet generated")
            
            
    def view_protected_attribute(self, ranked_df_top_k, selectProtectedAttributeToExamine_value):
        try:
            with self.viewAnalysisOut:
                protected = []
                protectedCount = []
                protectedPercentage = []
                colType = ranked_df_top_k[self.selectYAttribute.value].dtype
        
                if (colType != str and colType != object):
                    yMin = self.ranked_df[self.selectYAttribute.value].min()
                    yMean = self.ranked_df[self.selectYAttribute.value].mean()
                    yMax = self.ranked_df[self.selectYAttribute.value].max()
                    
                    #As it is -value in this case - - = +
                    if (yMin < 0):
                        adjust = yMin
                        yMin = yMin - adjust
                        yMean = yMean - adjust
                        yMax = yMax - adjust
                        
                    bins = [yMin, yMean, yMax]
            
                    yMin = round(yMin, 3)
                    yMean = round(yMean, 3)
                    yMax = round(yMax, 3)
        
                    newColName = self.selectYAttribute.value + "-range" 
                    names = ['', '']
            
    
    
                    if self.selectImpact.value == 0:
                        names = ['Neg_Outcome', 'Pos_Outcome']
                    #min to mean is neg, mean to max is pos
                    if self.selectImpact.value == 1:
                        names = ['Pos_Outcome', 'Neg_Outcome']
                    #min to mean is pos, mean to max is neg


            
                    ranked_df_top_k[newColName] = pd.cut(ranked_df_top_k[self.selectYAttribute.value], bins, labels=names).copy()
                    # if relevance score is a negative value add the minimum value to all values to make positive
                    print("")
                   
        except: 
            display("content not yet loaded")
                
        try:
            with self.viewProtectedPandasPlotA:
                clear_output(wait=True)
                feature = selectProtectedAttributeToExamine_value
                protected.append(feature)
                # protectedCount and protectedPercentage are array of type pd.Series
                protectedCount.append(ranked_df_top_k[feature].value_counts())
                protectedPercentage.append(ranked_df_top_k[feature].value_counts(normalize=True) * 100)
                count = 0
                if (len(protected) > 0):

                    for (PDSeries, PDSeriesPcnt, PDName) in zip(protectedCount, protectedPercentage, protected):
                            #Create a Dataframe to display the attribute stats
                        df = pd.DataFrame(dict(Count=PDSeries, Percentage=PDSeriesPcnt)).reset_index()
                        df = df.rename(columns={"index": PDName.capitalize()})
                        df = df.style.set_properties(**{'background-color': 'black',
                                                            'color': 'lawngreen',
                                                            'border-color': 'white'})

                        #Create a Dataframe to display the attribute stats
                        display(HTML("<b><font color='green'>Analysis of '" + PDName + "' in reduced ranked list:</b></font>"))
                        display(HTML(names[0] + "-> Selected for list AND above average relevance score (based on entire data set) between  " + str(yMin) + " and " + str(yMean) ), 
                                HTML(names[1] + "-> Selected for list BUT below average relevance score (based on entire data set) between " + str(yMean) + " and " + str(yMax)) )
                    
                        result = self.group_names[selectProtectedAttributeToExamine_value]
                        display(result)
                        display(df)
                        #Create two pandas plots and display them with matplot
                        plt.tight_layout();
                        fig, (ax, ax2) = plt.subplots(ncols=2, sharey=False);
                        fig.set_figheight(5);
                        fig.set_figwidth(10);
                        fig.subplots_adjust(wspace=0.4, bottom=0.3);
                        plotA = PDSeries.plot.pie(title="Representation in reduced list",ax=ax,autopct='%.2f');
                        plotB = pd.crosstab(ranked_df_top_k[PDName], ranked_df_top_k[newColName]).plot.barh(stacked=False, title="Distribution of outcome in reduced list",ax=ax2);
                        plt.show();



        except:
            print ("output not generated")
                
        if (colType == str or colType == object):
            print("Output label is not a number and should be converted")
     
 

        
    def render(self):
        space=widgets.Label('  ', layout=widgets.Layout(width='100%'))

        uploadFileHTML=widgets.HTML(
            "<h3><left>1. Upload Ranked list:</left></h3>")

        setRankedOutputHTML=widgets.HTML(
            "<h3><left>2. Identify the ranked output column:</left></h3>")

        effectOfRankingHTML=widgets.HTML(
            "<h4><left>Set the effect of ranking on the individual or group</left></h4>")

        setProtectedAttributesHTML=widgets.HTML(
            "<h3><left>3. Identify the protected attributes column:</left></h3>")

        visualiseProtectedAttributesHTML=widgets.HTML(
            "<h3><left>4. Visualise protected attributes:</left></h3>")

        analyzeProtectedHTML=widgets.HTML(
            "<h3><left>5. Analyze ranking for fairness:</left></h3>")

        analyzeProtectedHTML_B=widgets.HTML(
            "<h4><left>Specify the number of results that will receive attention:</left></h4>")

        analyzeProtectedHTML_C=widgets.HTML(
            "<h4><left>Specify the protected attribute to analyse:</left></h4>")
        
        resultsHTML=widgets.HTML("<h3><left>Results:</left></h3>")

        JoacmAndSinghHTML=widgets.HTML(
            "<h4><left>Joachims and Singh Fairness of Exposure in Ranking:</left></h4><h5>Is exposure fairly allocate based on relevance (regardless of how relevance is estimated)</h5>")
    
        FAIRHTML=widgets.HTML(
            "<h4><left>FA*IR: A Fair Top-k Ranking</left></h4><h5>Ranked group fairness: Does the representation of the protected group fall below a minimum proportion(p) at any point in the ranking (regardless of how relevance is estimated)</h5>")

        display(
            uploadFileHTML,
            self.fileUploader,
            self.outUploadData,
            space,
            setRankedOutputHTML,
            self.selectYAttribute,
            self.outSelectRankedListCol,
            space,
            effectOfRankingHTML,
            self.selectImpact,
            space,
            setProtectedAttributesHTML,
            self.selectProtectedAttributes,
            self.outSelectProtected,
            space,
            visualiseProtectedAttributesHTML,
            self.viewProtectedOut,

            space,

            analyzeProtectedHTML,

            # view_analysis input fields
            widgets.VBox([
                self.viewListInfoOut,
                space,
                analyzeProtectedHTML_B,
                space,
                self.selectTopKResults,
                space,
                self.selectPercentageOfResults,
                space,
                self.percentOrTopK,
                space,
                analyzeProtectedHTML_C,
                space,
                self.selectProtectedAttributeToExamine,
                space,
                self.viewAnalysisGroupPercentInPop,
                space,
                self.selectFairnessGoal,
                space,
                resultsHTML,
                JoacmAndSinghHTML,
                space,
                self.selectExpDropMeasure,
                space,
                self.viewAnalysisOut,
                self.viewProtectedPandasPlotA,
                space,
                FAIRHTML,
                space,
                self.setSignificance,
                self.viewFairOutputA,
                space,
            ]))

In [ ]:
protected_attributes, renamed_groups, group_names = dpUI.get_protected()
high_ranking_positive = dpUI.high_positive ()
fairness = review_fairness_UI(ranked_df=ranked_df, protected_attributes=protected_attributes, renamed_groups=renamed_groups,high_ranking_positive=high_ranking_positive, group_names = group_names)
fairness.render()

# THE END

# OLD METHODS